# Lagrangian relaxation-based heuristic algorithm

In [ ]:
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
from itertools import combinations
import time
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import math
import copy
import numpy as np

## Input basic data

In [ ]:
# Define the number of stations, which is 7 here
num_of_stations = 7
# Define the number of sections, which is 6 here
num_of_sections = 6
# Pure running time of trains with different speeds in each section
pure_running_time = {
    1: [30, 26, 26, 26, 29, 20],  # Speed type 1: 160 km/h
    2: [39, 34, 34, 34, 38, 26]  # Speed type 2: 120 km/h
}
# Define the acceleration and deceleration additional time of a train.
DT = 2  # The acceleration additional time
AT = 3  # The deceleration additional time

# Define the station capacity
station_track_capacity = 5  # The number of tracks per station is limited to 5

# Define M
M = 1080
# Define the minimum maintenance window duration
MOT = 120
# Define the earliest start time for the maintenance window each day
MOT_e = 180  # The earliest start time is 21:00, converted to 180
# Define the lastest end time for the maintenance window each day
MOT_l = 720  # The latest end time is 06:00 (+1), converted to 720

# Define the minimum departure and arrival headway
haa = 6  # The minimum arrival headway is 6 min
hdd = 6  # The minimum departure headway is 6 min
# Define the minimum departure-arrival headway
hda = 2  # The minimum departure-arrival headway is 2 min
# Define the minimum arrival-departure headway between the actual trains and maintenance windows (virtual trains)
had_f = 7  # The minimum arrival-departure headway between the freight trains and maintenance windows is 7 min
had_p = 9  # The minimum arrival-departure headway between the passenger trains and maintenance windows is 9 min

# Define the unscheduling penalty cost for maintenance activities with different importance level
importance_penalty_costs = {
    1: 1000,  # 1 is "normal", the unscheduling penalty cost is set to 1000
    2: 2000,  # 2 is "important", the unscheduling penalty cost is set to 2000
    3: 3000  # 3 is "very important", the unscheduling penalty cost is set to 3000
}

Modify the number of maintenance activities that can be scheduled simultaneously here

In [ ]:
max_compatible_activities = 3  # A maximum of 3 maintenance activities can be scheduled simultaneously in each maintenance window

Modify the maintenance volume and its distribution

In [ ]:
balanced_maintenance_indicator = 1  # 1 means relatively balanced distribution

In [ ]:
maintenance_volume_indicator = 2

In [ ]:
if maintenance_volume_indicator == 1:  # 1 means the maintenance volume of 180, and each section has 30 maintenance activities
    if balanced_maintenance_indicator == 1:  # 1 means relatively balanced distribution
        maintenance_csv = 'balanced_180_maintenance_volume.csv'  # Read the file of 'balanced_180_maintenance_volume.csv'
    else:
        maintenance_csv = 'unbalanced_180_maintenance_volume.csv'  # Read the file of 'unbalanced_180_maintenance_volume.csv'
else:  # 2 means the maintenance volume of 240, and each section has 40 maintenance activities
    if balanced_maintenance_indicator == 1:  # 1 means relatively balanced distribution
        maintenance_csv = 'balanced_240_maintenance_volume.csv'  # Read the file of 'balanced_240_maintenance_volume.csv'
    else:
        maintenance_csv = 'unbalanced_240_maintenance_volume.csv'  # Read the file of 'balanced_240_maintenance_volume.csv'

In [ ]:
maintenance_df = pd.read_csv(maintenance_csv)

# Create a dictionary 'maintenance_activities_info' to store detailed information for each maintenance activity
maintenance_activities_info = {}  # The key is the activity number, and the value is a list containing the activity type, the start day of the optimal scheduling range, the end day of the optimal scheduling range, the fixed scheduling day, the duration, and the importance level.

for _, row in maintenance_df.iterrows():
    maintenance_activities_info[int(row['Activity ID'])] = [
        int(row['Activity Type']),
        int(row['Start Date']),
        int(row['End Date']),
        int(row['Fixed Scheduling Date']),
        int(row['Estimated Duration']),
        int(row['Importance Level'])
    ]

# Create a dictionary 'section_maintenance_activities' to store the activities for each section
section_maintenance_activities = {}

for section in range(1, num_of_sections + 1):
    section_maintenance_activities[section] = list(
        maintenance_df[maintenance_df['Section'] == section]['Activity ID']
    )

Modify the mutually exclusive relationship between maintenance activities here

In [ ]:
maintenance_mutually_exclusive_indicator = 2

In [ ]:
if maintenance_mutually_exclusive_indicator == 1:  # 1 means 29 mutually exclusive maintenance activity pairs
    mutually_exclusive_activities = {
        1: [16, 17, 2],
        3: [4, 8, 16, 17, 2],
        4: [3, 8, 16, 17],
        8: [3, 4, 16, 17],
        11: [16, 17, 14],
        12: [16, 17, 7],
        13: [16, 17, 15],
        15: [16, 17, 10, 13],
        16: [1, 3, 4, 8, 11, 12, 13, 15],
        17: [1, 3, 4, 8, 11, 12, 13, 15],
        2: [1, 3, 14],
        5: [6, 10],
        6: [5],
        7: [12],
        9: [10],
        10: [5, 9, 15],
        14: [2, 11]
    }  # The key is the activity type number, and the value is a list containing the activity type numbers that are mutually exclusive with the activity.
else:  # 2 means 39 mutually exclusive maintenance activity pairs
    mutually_exclusive_activities = {
        1: [16, 17, 2, 5],
        3: [4, 8, 16, 17, 2],
        4: [3, 8, 16, 17],
        8: [3, 4, 16, 17],
        11: [16, 17, 14, 9],
        12: [16, 17, 7, 15],
        13: [16, 17, 10, 15],
        15: [16, 17, 12, 13, 10],
        16: [1, 3, 4, 8, 11, 12, 13, 15],
        17: [1, 3, 4, 8, 11, 12, 13, 15, 14],
        2: [1, 3, 7, 14],
        5: [1, 6, 9, 10],
        6: [5, 7, 10],
        7: [2, 6, 9, 12],
        9: [5, 7, 10, 11],
        10: [5, 6, 9, 13, 15],
        14: [2, 11, 17]
    }

Modify the daily train volumes here

In [ ]:
num_of_trains = [30, 40, 50]  # The number of trains is 30, 40, 50 on day 1, 2, 3

In [ ]:
# Read the train data
# Build a function to read the train data from the 'train_data_XX.csv' file and return a dictionary
def read_train_data(num_trains):
    file_name = f'train_data_{num_trains}.csv'
    df = pd.read_csv(file_name)

    # Create an empty dictionary to store the processed data
    processed_data = {}

    for _, row in df.iterrows():
        departure_range = eval(row['Departure Time Range'])
        stop_scheme = [int(float(x)) for x in eval(row['Stop Scheme'])]

        # Store the processed data into the dictionary 'processed_data'
        processed_data[int(row['Train ID'])] = [
            int(row['Train Type']),
            int(row['Speed Type']),
            departure_range,
            stop_scheme
        ]

    return processed_data


train_info = {}
# Read the train data in the corresponding .csv file according to the daily train volume
for day, num_trains in enumerate(num_of_trains, 1):
    train_info[day] = read_train_data(num_trains)

Generate passenger train timetables

In [ ]:
# The departure time list of passenger trains on each day
passenger_train_departure_times = [30, 90, 150, 210, 615, 645, 675, 705]  # Converted to min
# The dwell time list of passenger trains on each day
passenger_train_dwell_times = [0, 0, 8, 0, 15]  # min
# Create a dictionary to record the arrival and departure time of passenger trains at each station
station_dep_arr_times = {}

for i in range(len(passenger_train_departure_times)):
    arr_times = []  # Store arrival time
    dep_times = []  # Store departure time
    for j in range(num_of_stations):
        # Calculate the arrival and departure time
        if j == 0:  # The train only has the departure time at the start station
            dep_time = passenger_train_departure_times[i]
            dep_times.append(dep_time)
        elif j != num_of_stations - 1:
            if j == 1:
                if passenger_train_dwell_times[j - 1] == 0:
                    arr_time = dep_times[j - 1] + pure_running_time[1][j - 1] + DT
                    dep_time = arr_time
                else:
                    arr_time = dep_times[j - 1] + pure_running_time[1][j - 1] + DT + AT
                    dep_time = arr_time
            else:
                if passenger_train_dwell_times[j - 1] == 0 and passenger_train_dwell_times[
                    j - 2] == 0:
                    arr_time = dep_times[j - 1] + pure_running_time[1][j - 1]
                    dep_time = arr_time
                elif passenger_train_dwell_times[j - 1] != 0 and passenger_train_dwell_times[
                    j - 2] == 0:
                    arr_time = dep_times[j - 1] + pure_running_time[1][j - 1] + AT
                    dep_time = arr_time + passenger_train_dwell_times[j - 1]
                elif passenger_train_dwell_times[j - 1] == 0 and passenger_train_dwell_times[
                    j - 2] != 0:
                    arr_time = dep_times[j - 1] + pure_running_time[1][j - 1] + DT
                    dep_time = arr_time
                else:
                    arr_time = dep_times[j - 1] + pure_running_time[1][j - 1] + DT + AT
                    dep_time = arr_time + passenger_train_dwell_times[j - 1]
            arr_times.append(arr_time)
            dep_times.append(dep_time)
        else:  # The train only has the arrival time at the start station
            if passenger_train_dwell_times[j - 2] == 0:
                arr_time = dep_times[j - 1] + pure_running_time[1][j - 1] + AT
            else:
                arr_time = dep_times[j - 1] + pure_running_time[1][j - 1] + DT + AT
            arr_times.append(arr_time)

        # Fill the dictionary 'station_dep_arr_times', the value of which is a list of departure times and arrival times. The departure time is related to the first station to the second to last station, and the arrival time is related to the second station to the last station.
        station_dep_arr_times[passenger_train_departure_times[i]] = [dep_times, arr_times]

Define the objective function weight combination here

In [ ]:
w1 = 0.5
w2 = 0.5

Initialize the Lagrange multipliers here

In [ ]:
lagrangian_multipliers = {}

for day in range(1, len(num_of_trains) + 1):
    lagrangian_multipliers[day] = {}

    for section_id, maintenance_activities in section_maintenance_activities.items():
        lagrangian_multipliers[day][section_id] = {}
        for activity_id in maintenance_activities:
            lagrangian_multipliers[day][section_id][activity_id] = {}
            for i in [1, 2]:  # i = 1 indicates the Lagrangian multiplier associated with Constraint (21), and i = 2 indicates the Lagrangian multiplier associated with Constraint (22)
                # Initialize the Lagrangian multipliers to 0.0
                lagrangian_multipliers[day][section_id][activity_id][i] = 0.0

## Construct the subproblem models

### Construct the day-specific subproblem models related to train timetabling with maintenance window setting

In [ ]:
# Manage the daily subproblem model in the dictionary 'train_timetables_with_maintenance_windows_subproblem_models', where the key is the day and the value is the model object.
train_timetables_with_maintenance_windows_subproblem_models = {}

for day in range(1, len(num_of_trains) + 1):
    train_timetables_with_maintenance_windows_subproblem_models[day] = gp.Model(
        f'Train_timetables_with_maintenance_windows_day_{day}_with_train_volume_{num_of_trains[day - 1]}')

#### Construct variables

In [ ]:
dep_time_vars_of_actual_trains = {}  # Departure time variables of actual trains on each day
arr_time_vars_of_actual_trains = {}  # Arrival time variables of actual trains on each day
dwell_time_vars_of_actual_trains = {}  # Dwell time variables of actual trains on each day
stop_indicator_vars_of_actual_trains = {}  # Stop indicator variables of actual trains on each day

for day, day_train_info in train_info.items():
    dep_time_vars_of_actual_trains[day] = {}
    arr_time_vars_of_actual_trains[day] = {}
    dwell_time_vars_of_actual_trains[day] = {}
    stop_indicator_vars_of_actual_trains[day] = {}

    for train_id, train_data in day_train_info.items():
        # Obtain train data
        train_type = train_data[0]  # train type
        speed_type = train_data[1]  # train speed type
        train_departure_range = train_data[2]  # departure time window
        train_stop_plan = train_data[3]  # stop plan

        # If it is a passenger train, it only has the arrival and departure time at each station, which is fixed. We can define it by setting upper and lower limits directly when constructing the variables.
        if train_type == 1:
            train_departure_time = train_departure_range[0]
            for i in range(1, num_of_stations + 1):
                if i == 1:
                    dep_time_vars_of_actual_trains[day][(train_id, i)] = \
                        train_timetables_with_maintenance_windows_subproblem_models[day].addVar(
                            lb=station_dep_arr_times[train_departure_time][0][i - 1],
                            ub=station_dep_arr_times[train_departure_time][0][i - 1],
                            vtype=GRB.INTEGER,
                            name=f'dep_time_var_day_{day}_train_{train_id}_station_{i}'
                        )
                elif i != num_of_stations:
                    arr_time_vars_of_actual_trains[day][(train_id, i)] = \
                        train_timetables_with_maintenance_windows_subproblem_models[day].addVar(
                            lb=station_dep_arr_times[train_departure_time][1][i - 2],
                            ub=station_dep_arr_times[train_departure_time][1][i - 2],
                            vtype=GRB.INTEGER,
                            name=f'arr_time_var_day_{day}_train_{train_id}_station_{i}'
                        )
                    dep_time_vars_of_actual_trains[day][(train_id, i)] = \
                        train_timetables_with_maintenance_windows_subproblem_models[day].addVar(
                            lb=station_dep_arr_times[train_departure_time][0][i - 1],
                            ub=station_dep_arr_times[train_departure_time][0][i - 1],
                            vtype=GRB.INTEGER,
                            name=f'dep_time_var_day_{day}_train_{train_id}_station_{i}'
                        )
                else:
                    arr_time_vars_of_actual_trains[day][(train_id, i)] = \
                        train_timetables_with_maintenance_windows_subproblem_models[day].addVar(
                            lb=station_dep_arr_times[train_departure_time][1][i - 2],
                            ub=station_dep_arr_times[train_departure_time][1][i - 2],
                            vtype=GRB.INTEGER,
                            name=f'arr_time_var_day_{day}_train_{train_id}_station_{i}'
                        )
        # If it is a freight train, all four types of variables above need to be set
        else:
            for i in range(1, num_of_stations + 1):
                if i == 1:
                    dep_time_vars_of_actual_trains[day][(train_id, i)] = \
                        train_timetables_with_maintenance_windows_subproblem_models[day].addVar(
                            lb=train_departure_range[0],
                            ub=train_departure_range[1],
                            vtype=GRB.INTEGER,
                            name=f'dep_time_var_day_{day}_train_{train_id}_station_{i}'
                        )
                elif i != num_of_stations:
                    arr_time_vars_of_actual_trains[day][(train_id, i)] = \
                        train_timetables_with_maintenance_windows_subproblem_models[day].addVar(
                            lb=train_departure_range[0],
                            ub=1080,
                            vtype=GRB.INTEGER,
                            name=f'arr_time_var_day_{day}_train_{train_id}_station_{i}'
                        )
                    dep_time_vars_of_actual_trains[day][(train_id, i)] = \
                        train_timetables_with_maintenance_windows_subproblem_models[day].addVar(
                            lb=train_departure_range[0],
                            ub=1080,
                            vtype=GRB.INTEGER,
                            name=f'dep_time_var_day_{day}_train_{train_id}_station_{i}'
                        )
                    dwell_time_vars_of_actual_trains[day][(train_id, i)] = \
                        train_timetables_with_maintenance_windows_subproblem_models[day].addVar(
                            lb=train_stop_plan[i - 2],
                            ub=60,  # The maximum dwell time is set to 60 min
                            vtype=GRB.INTEGER,
                            name=f'dwell_time_var_day_{day}_train_{train_id}_station_{i}'
                        )
                    stop_indicator_vars_of_actual_trains[day][(train_id, i)] = \
                        train_timetables_with_maintenance_windows_subproblem_models[day].addVar(
                            vtype=GRB.BINARY,
                            name=f'stop_indicator_var_day_{day}_train_{train_id}_station_{i}'
                        )
                else:
                    arr_time_vars_of_actual_trains[day][(train_id, i)] = \
                        train_timetables_with_maintenance_windows_subproblem_models[day].addVar(
                            lb=train_departure_range[0],
                            ub=1080,
                            vtype=GRB.INTEGER,
                            name=f'arr_time_var_day_{day}_train_{train_id}_station_{i}'
                        )

In [ ]:
start_time_vars_of_maintenance_windows = {}  # Start time variables of maintenance windows on each day
end_time_vars_of_maintenance_windows = {}  # End time variables of maintenance windows on each day

for day in train_info.keys():
    start_time_vars_of_maintenance_windows[day] = {}
    end_time_vars_of_maintenance_windows[day] = {}

    for i in range(1, num_of_sections + 1):
        start_time_vars_of_maintenance_windows[day][i] = train_timetables_with_maintenance_windows_subproblem_models[
            day].addVar(
            lb=MOT_e,
            ub=MOT_l,
            vtype=GRB.INTEGER,
            name=f'maintenance_window_start_time_var_day_{day}_section_{i}'
        )
        end_time_vars_of_maintenance_windows[day][i] = train_timetables_with_maintenance_windows_subproblem_models[
            day].addVar(
            lb=MOT_e,
            ub=MOT_l,
            vtype=GRB.INTEGER,
            name=f'maintenance_window_end_time_var_day_{day}_section_{i}'
        )

In [ ]:
dep_seq_vars_of_actual_trains = {}  # The actual train departure sequence variables for each day
arr_seq_vars_of_actual_trains = {}  # The actual train arrival sequence variables for each day

for day, day_train_info in train_info.items():
    dep_seq_vars_of_actual_trains[day] = {}
    arr_seq_vars_of_actual_trains[day] = {}

    for train_id_1 in day_train_info.keys():
        for train_id_2 in day_train_info.keys():
            if train_id_1 < train_id_2:  # Only set once for each pair of trains
                for i in range(1, num_of_sections + 1):
                    dep_seq_vars_of_actual_trains[day][(train_id_1, train_id_2, i)] = \
                        train_timetables_with_maintenance_windows_subproblem_models[day].addVar(
                            vtype=GRB.BINARY,
                            name=f'dep_seq_var_day_{day}_trains_{train_id_1}_{train_id_2}_section_{i}'
                        )
                    arr_seq_vars_of_actual_trains[day][(train_id_1, train_id_2, i)] = \
                        train_timetables_with_maintenance_windows_subproblem_models[day].addVar(
                            vtype=GRB.BINARY,
                            name=f'arr_seq_var_day_{day}_trains_{train_id_1}_{train_id_2}_section_{i}'
                        )

In [ ]:
maintenance_window_train_seq_vars = {}  # Sequence variables between maintenance windows and actual trains for each day

for day, day_train_info in train_info.items():
    maintenance_window_train_seq_vars[day] = {}

    for train_id in day_train_info.keys():
        for i in range(1, num_of_sections + 1):
            maintenance_window_train_seq_vars[day][(train_id, i)] = \
                train_timetables_with_maintenance_windows_subproblem_models[day].addVar(
                    vtype=GRB.BINARY,
                    name=f'maintenance_window_train_seq_var_day_{day}_train_{train_id}_section_{i}'
                )

In [ ]:
train_meeting_vars = {}  # The variables related to train meeting for each day
train_arr_dep_seq_vars = {}  # Train arrival and departure sequence variables at stations for each day

for day, day_train_info in train_info.items():
    train_meeting_vars[day] = {}
    train_arr_dep_seq_vars[day] = {}

    for train_id_1 in day_train_info.keys():
        for train_id_2 in day_train_info.keys():
            if train_id_1 < train_id_2:  # Only set once for each pair of trains
                for i in range(2, num_of_stations):
                    train_meeting_vars[day][(train_id_1, train_id_2, i)] = \
                        train_timetables_with_maintenance_windows_subproblem_models[day].addVar(
                            vtype=GRB.BINARY,
                            name=f'train_meeting_var_day_{day}_trains_{train_id_1}_{train_id_2}_station_{i}'
                        )
                    train_arr_dep_seq_vars[day][(train_id_1, train_id_2, i)] = \
                        train_timetables_with_maintenance_windows_subproblem_models[day].addVar(
                            vtype=GRB.BINARY,
                            name=f'train_arr_dep_seq_var_day_{day}_trains_{train_id_1}_{train_id_2}_station_{i}'
                        )
                    train_arr_dep_seq_vars[day][(train_id_2, train_id_1, i)] = \
                        train_timetables_with_maintenance_windows_subproblem_models[day].addVar(
                            vtype=GRB.BINARY,
                            name=f'train_arr_dep_seq_var_day_{day}_trains_{train_id_2}_{train_id_1}_station_{i}'
                        )

#### Construct constraints

In [ ]:
arr_dep_times_cons_of_freight_trains = {}  # The constraints related to the arrival and departure time of freight trains at each station on each day
stop_indicator_cons_of_freight_trains = {}  # The constraints related to the stop indicator of freight trains at each station on each day

for day, dwell_time_vars in dwell_time_vars_of_actual_trains.items():
    arr_dep_times_cons_of_freight_trains[day] = {}
    stop_indicator_cons_of_freight_trains[day] = {}

    for (train_id, station_id), dwell_time_var in dwell_time_vars.items():
        arr_time_var = arr_time_vars_of_actual_trains[day][(train_id, station_id)]
        dep_time_var = dep_time_vars_of_actual_trains[day][(train_id, station_id)]
        arr_dep_times_cons_of_freight_trains[day][(train_id, station_id)] = \
            train_timetables_with_maintenance_windows_subproblem_models[day].addConstr(
                arr_time_var + dwell_time_var == dep_time_var,
                name=f'arr_dep_times_con_day_{day}_train_{train_id}_station_{station_id}'
            )
        stop_indicator_var = stop_indicator_vars_of_actual_trains[day][(train_id, station_id)]
        stop_indicator_cons_of_freight_trains[day][(train_id, station_id)] = \
            train_timetables_with_maintenance_windows_subproblem_models[day].addConstr(
                dwell_time_var <= M * stop_indicator_var,
                name=f'stop_indicator_con_day_{day}_train_{train_id}_station_{station_id}'
            )

In [ ]:
min_section_running_time_cons_of_freight_trains = {}  # The constraints related to the minimum running time of freight trains at each section on each day

for day, day_train_info in train_info.items():
    min_section_running_time_cons_of_freight_trains[day] = {}

    for train_id, train_data in day_train_info.items():
        train_type = train_data[0]
        speed_type = train_data[1]
        if train_type == 1:  # Continue the passenger trains
            continue
        for section_id in range(1, num_of_sections + 1):
            arr_time_var = arr_time_vars_of_actual_trains[day][(train_id, section_id + 1)]
            dep_time_var = dep_time_vars_of_actual_trains[day][(train_id, section_id)]
            pure_run_time = pure_running_time[speed_type][section_id - 1]
            if section_id == 1:
                stop_indicator_var = stop_indicator_vars_of_actual_trains[day][
                    (train_id, section_id + 1)]
                min_section_running_time_cons_of_freight_trains[day][(train_id, section_id)] = \
                    train_timetables_with_maintenance_windows_subproblem_models[day].addConstr(
                        arr_time_var - dep_time_var >= pure_run_time + DT + AT * stop_indicator_var,
                        name=f'min_section_running_time_con_day_{day}_train_{train_id}_section_{section_id}'
                    )
            elif section_id == num_of_sections:
                stop_indicator_var = stop_indicator_vars_of_actual_trains[day][
                    (train_id, section_id)]
                min_section_running_time_cons_of_freight_trains[day][(train_id, section_id)] = \
                    train_timetables_with_maintenance_windows_subproblem_models[day].addConstr(
                        arr_time_var - dep_time_var >= pure_run_time + DT * stop_indicator_var + AT,
                        name=f'min_section_running_time_con_day_{day}_train_{train_id}_section_{section_id}'
                    )
            else:
                stop_indicator_var_1 = stop_indicator_vars_of_actual_trains[day][
                    (train_id, section_id)]
                stop_indicator_var_2 = stop_indicator_vars_of_actual_trains[day][
                    (train_id, section_id + 1)]
                min_section_running_time_cons_of_freight_trains[day][(train_id, section_id)] = \
                    train_timetables_with_maintenance_windows_subproblem_models[day].addConstr(
                        arr_time_var - dep_time_var >= pure_run_time + DT * stop_indicator_var_1 + AT * stop_indicator_var_2,
                        name=f'min_section_running_time_con_day_{day}_train_{train_id}_section_{section_id}'
                    )

In [ ]:
min_maintenance_window_duration_cons = {}  # The minimum maintenance window duration requirement constraint

for day in start_time_vars_of_maintenance_windows.keys():
    min_maintenance_window_duration_cons[day] = {}

    for section_id in range(1, num_of_sections + 1):
        start_time_var = start_time_vars_of_maintenance_windows[day][section_id]
        end_time_var = end_time_vars_of_maintenance_windows[day][section_id]
        min_maintenance_window_duration_cons[day][section_id] = \
            train_timetables_with_maintenance_windows_subproblem_models[day].addConstr(
                end_time_var - start_time_var >= MOT,
                name=f'min_maintenance_window_duration_con_day_{day}_section_{section_id}'
            )

In [ ]:
dep_time_seq_cons_of_actual_trains = {}  # The actual train departure time sequence constraints for each day

for day, dep_seq_vars in dep_seq_vars_of_actual_trains.items():
    dep_time_seq_cons_of_actual_trains[day] = {}

    for (train_id_1, train_id_2, section_id), dep_seq_var in dep_seq_vars.items():
        dep_time_var_1 = dep_time_vars_of_actual_trains[day][(train_id_1, section_id)]
        dep_time_var_2 = dep_time_vars_of_actual_trains[day][(train_id_2, section_id)]
        # If Train 2 departs after Train 1, the departure time of Train 2 must be greater than or equal to the departure time of Train 1 plus the minimum departure headway.
        dep_time_seq_cons_of_actual_trains[day][(train_id_1, train_id_2, section_id, 1)] = \
            train_timetables_with_maintenance_windows_subproblem_models[day].addConstr(
                dep_time_var_2 + M * (1 - dep_seq_var) >= dep_time_var_1 + hdd,
                name=f'dep_time_seq_con_day_{day}_trains_{train_id_1}_{train_id_2}_section_{section_id}_1'
            )
        # If Train 1 departs after Train 2, the departure time of Train 1 must be greater than or equal to the departure time of Train 2 plus the minimum departure headway.
        dep_time_seq_cons_of_actual_trains[day][(train_id_1, train_id_2, section_id, 2)] = \
            train_timetables_with_maintenance_windows_subproblem_models[day].addConstr(
                dep_time_var_1 + M * dep_seq_var >= dep_time_var_2 + hdd,
                name=f'dep_time_seq_con_day_{day}_trains_{train_id_1}_{train_id_2}_section_{section_id}_2'
            )

In [ ]:
arr_time_seq_cons_of_actual_trains = {}  # The actual train arrival time sequence constraints for each day

for day, arr_seq_vars in arr_seq_vars_of_actual_trains.items():
    arr_time_seq_cons_of_actual_trains[day] = {}

    for (train_id_1, train_id_2, section_id), arr_seq_var in arr_seq_vars.items():
        arr_time_var_1 = arr_time_vars_of_actual_trains[day][(train_id_1, section_id + 1)]
        arr_time_var_2 = arr_time_vars_of_actual_trains[day][(train_id_2, section_id + 1)]
        # If Train 2 arrives after Train 1, the arrival time of Train 2 must be greater than or equal to the arrival time of Train 1 plus the minimum arrival headway.
        arr_time_seq_cons_of_actual_trains[day][(train_id_1, train_id_2, section_id, 1)] = \
            train_timetables_with_maintenance_windows_subproblem_models[day].addConstr(
                arr_time_var_2 + M * (1 - arr_seq_var) >= arr_time_var_1 + haa,
                name=f'arr_time_seq_con_day_{day}_trains_{train_id_1}_{train_id_2}_section_{section_id}_1'
            )
        # If Train 1 arrives after Train 2, the arrival time of Train 1 must be greater than or equal to the arrival time of Train 2 plus the minimum arrival headway.
        arr_time_seq_cons_of_actual_trains[day][(train_id_1, train_id_2, section_id, 2)] = \
            train_timetables_with_maintenance_windows_subproblem_models[day].addConstr(
                arr_time_var_1 + M * arr_seq_var >= arr_time_var_2 + haa,
                name=f'arr_time_seq_con_day_{day}_trains_{train_id_1}_{train_id_2}_section_{section_id}_2'
            )

In [ ]:
consistent_arr_dep_seq_cons_of_actual_trains = {}  # Consistency constraints on actual train arrival and departure sequence

for day, arr_seq_vars in arr_seq_vars_of_actual_trains.items():
    consistent_arr_dep_seq_cons_of_actual_trains[day] = {}

    for (train_id_1, train_id_2, section_id), arr_seq_var in arr_seq_vars.items():
        dep_seq_var = dep_seq_vars_of_actual_trains[day][(train_id_1, train_id_2, section_id)]
        # The relevant departure sequence variable and the arrival sequence variable must both be 0 or 1.
        consistent_arr_dep_seq_cons_of_actual_trains[day][(train_id_1, train_id_2, section_id)] = \
            train_timetables_with_maintenance_windows_subproblem_models[day].addConstr(
                arr_seq_var == dep_seq_var,
                name=f'consistent_arr_dep_seq_con_day_{day}_trains_{train_id_1}_{train_id_2}_section_{section_id}_1'
            )

In [ ]:
maintenance_window_train_seq_cons = {}  # The sequence relationship constraint between the maintenance windows and the actual trains on each day

for day in maintenance_window_train_seq_vars.keys():
    day_train_info = train_info[day]
    maintenance_window_train_seq_cons[day] = {}

    for (train_id, section_id), seq_var in maintenance_window_train_seq_vars[day].items():
        train_type = day_train_info[train_id][0]
        dep_time_var = dep_time_vars_of_actual_trains[day][(train_id, section_id)]
        arr_time_var = arr_time_vars_of_actual_trains[day][(train_id, section_id + 1)]
        start_time_var = start_time_vars_of_maintenance_windows[day][section_id]
        end_time_var = end_time_vars_of_maintenance_windows[day][section_id]
        # If the actual train is a passenger train
        if train_type == 1:
            # If the train is after the maintenance window, the departure time of the train at the starting station of the section must be greater than or equal to the end time of the maintenance window plus the minimum arrival-departure headway.
            maintenance_window_train_seq_cons[day][(train_id, section_id, 1)] = \
                train_timetables_with_maintenance_windows_subproblem_models[day].addConstr(
                    dep_time_var + M * (1 - seq_var) >= end_time_var + had_p,
                    name=f'maintenance_window_train_seq_con_day_{day}_train_{train_id}_section_{section_id}_1'
                )
            # If the maintenance window is after the train, the start time of the maintenance window must be greater than or equal to the arrival time of the train at the end station of the section plus the minimum arrival-departure headway.
            maintenance_window_train_seq_cons[day][(train_id, section_id, 2)] = \
                train_timetables_with_maintenance_windows_subproblem_models[day].addConstr(
                    start_time_var + M * seq_var >= arr_time_var + had_p,
                    name=f'maintenance_window_train_seq_con_day_{day}_train_{train_id}_section_{section_id}_2'
                )
        # If the actual train is a freight train
        else:
            maintenance_window_train_seq_cons[day][(train_id, section_id, 1)] = \
                train_timetables_with_maintenance_windows_subproblem_models[day].addConstr(
                    dep_time_var + M * (1 - seq_var) >= end_time_var + had_f,
                    name=f'maintenance_window_train_seq_con_day_{day}_train_{train_id}_section_{section_id}_1'
                )
            maintenance_window_train_seq_cons[day][(train_id, section_id, 2)] = \
                train_timetables_with_maintenance_windows_subproblem_models[day].addConstr(
                    start_time_var + M * seq_var >= arr_time_var + had_f,
                    name=f'maintenance_window_train_seq_con_day_{day}_train_{train_id}_section_{section_id}_2'
                )

#### Construct object function

At each iteration, the objective function needs to be updated according to the updated Lagrangian multipliers

In [ ]:
def define_or_update_obj_of_train_timetables_with_maintenance_windows_subproblem_models():
    for inner_day, inner_day_train_info in train_info.items():
        obj_of_daily_train_timetables_with_maintenance_windows_subproblem = gp.LinExpr(0)  # Initialize the objective function for each day
        for inner_train_id, inner_train_data in inner_day_train_info.items():
            inner_train_type = inner_train_data[0]
            if inner_train_type == 1: continue
            inner_dep_time_var = dep_time_vars_of_actual_trains[inner_day][(inner_train_id, 1)]
            inner_arr_time_var = arr_time_vars_of_actual_trains[inner_day][(inner_train_id, num_of_stations)]
            obj_of_daily_train_timetables_with_maintenance_windows_subproblem += w1 * (
                    inner_arr_time_var - inner_dep_time_var)
        for inner_section_id, inner_maintenance_activities in section_maintenance_activities.items():
            inner_start_time_var = start_time_vars_of_maintenance_windows[inner_day][inner_section_id]
            inner_end_time_var = end_time_vars_of_maintenance_windows[inner_day][inner_section_id]
            for inner_activity_id in inner_maintenance_activities:
                inner_lagrangian_multiplier_of_start = \
                    lagrangian_multipliers[inner_day][inner_section_id][inner_activity_id][1]
                inner_lagrangian_multiplier_of_end = \
                    lagrangian_multipliers[inner_day][inner_section_id][inner_activity_id][2]
                obj_of_daily_train_timetables_with_maintenance_windows_subproblem += inner_lagrangian_multiplier_of_start * inner_start_time_var - inner_lagrangian_multiplier_of_end * inner_end_time_var
        daily_train_timetables_with_maintenance_windows_subproblem_model = \
            train_timetables_with_maintenance_windows_subproblem_models[inner_day]
        daily_train_timetables_with_maintenance_windows_subproblem_model.setObjective(
            obj_of_daily_train_timetables_with_maintenance_windows_subproblem, GRB.MINIMIZE)  # The objective function is to minimize
        daily_train_timetables_with_maintenance_windows_subproblem_model.update()  # Update model

### Construct the section-specific subproblem models related to maintenance scheduling within the planning horizon

In [ ]:
# The maintenance scheduling subproblem model for each section is managed in the dictionary 'maintenance_planning_subproblem_models', where the key is the section and the value is the model object.
maintenance_planning_subproblem_models = {}
for section_id in range(1, num_of_sections + 1):
    maintenance_planning_subproblem_models[section_id] = gp.Model(f'Maintenance_planning_section_{section_id}')

#### Construct variables

In [ ]:
maintenance_activity_scheduling_day_vars = {}  # Maintenance activity scheduling day selection variable

for section_id, maintenance_activities in section_maintenance_activities.items():
    maintenance_activity_scheduling_day_vars[section_id] = {}

    for day in range(1, len(num_of_trains) + 1):
        for activity_id in maintenance_activities:
            maintenance_activity_scheduling_day_vars[section_id][(day, activity_id)] = \
                maintenance_planning_subproblem_models[section_id].addVar(
                    vtype=GRB.BINARY,
                    name=f'maintenance_activity_scheduling_day_var_section_{section_id}_day_{day}_activity_{activity_id}'
                )

In [ ]:
maintenance_activity_start_time_vars = {}  # Maintenance activity start time variable
maintenance_activity_end_time_vars = {}  # Maintenance activity end time variable

for section_id, maintenance_activities in section_maintenance_activities.items():
    maintenance_activity_start_time_vars[section_id] = {}
    maintenance_activity_end_time_vars[section_id] = {}

    for day in range(1, len(num_of_trains) + 1):
        for activity_id in maintenance_activities:
            maintenance_activity_start_time_vars[section_id][(day, activity_id)] = \
                maintenance_planning_subproblem_models[section_id].addVar(
                    lb=0,
                    ub=720,
                    vtype=GRB.INTEGER,
                    name=f'maintenance_activity_start_time_var_section_{section_id}_day_{day}_activity_{activity_id}'
                )
            maintenance_activity_end_time_vars[section_id][(day, activity_id)] = maintenance_planning_subproblem_models[
                section_id].addVar(
                lb=0,
                ub=720,
                vtype=GRB.INTEGER,
                name=f'maintenance_activity_end_time_var_section_{section_id}_day_{day}_activity_{activity_id}'
            )

In [ ]:
maintenance_activities_in_same_day_indicator_vars = {}  # The variable indicating whether the maintenance activities are in the same maintenance window

for section_id, maintenance_activities in section_maintenance_activities.items():
    maintenance_activities_in_same_day_indicator_vars[section_id] = {}

    for activity_id_1 in maintenance_activities:
        for activity_id_2 in maintenance_activities:
            if activity_id_1 < activity_id_2:  # Only set once per pair of acticities
                start_date_1 = maintenance_activities_info[activity_id_1][1]
                end_date_1 = maintenance_activities_info[activity_id_1][2]
                days_in_range_1 = list(range(start_date_1, end_date_1 + 1))
                start_date_2 = maintenance_activities_info[activity_id_2][1]
                end_date_2 = maintenance_activities_info[activity_id_2][2]
                days_in_range_2 = list(range(start_date_2, end_date_2 + 1))
                # Take the intersection of two day lists to get the common day range
                common_days = set(days_in_range_1) & set(days_in_range_2)
                for day in common_days:
                    maintenance_activities_in_same_day_indicator_vars[section_id][
                        (day, activity_id_1, activity_id_2)] = maintenance_planning_subproblem_models[
                        section_id].addVar(
                        vtype=GRB.BINARY,
                        name=f'maintenance_activities_in_same_day_indicator_var_section_{section_id}_day_{day}_activities_{activity_id_1}_{activity_id_2}'
                    )

In [ ]:
maintenance_activities_seq_vars = {}  # Sequence variables between maintenance activities

for section_id, maintenance_activities in section_maintenance_activities.items():
    maintenance_activities_seq_vars[section_id] = {}

    for activity_id_1 in maintenance_activities:
        activity_type_1 = maintenance_activities_info[activity_id_1][0]
        # If activity_type_1 is in the mutually exclusive activity relationship dictionary, only activities that are mutually exclusive with it are considered.
        if activity_type_1 in mutually_exclusive_activities:
            for activity_id_2 in maintenance_activities:
                if activity_id_2 == activity_id_1:
                    continue
                activity_type_2 = maintenance_activities_info[activity_id_2][0]
                # If activity_type_2 is mutually exclusive with activity_type_1
                if activity_type_2 in mutually_exclusive_activities[activity_type_1]:
                    start_date_1 = maintenance_activities_info[activity_id_1][1]
                    end_date_1 = maintenance_activities_info[activity_id_1][2]
                    days_in_range_1 = list(range(start_date_1, end_date_1 + 1))
                    start_date_2 = maintenance_activities_info[activity_id_2][1]
                    end_date_2 = maintenance_activities_info[activity_id_2][2]
                    days_in_range_2 = list(range(start_date_2, end_date_2 + 1))
                    # Take the intersection of two day lists to get the common day range
                    common_days = set(days_in_range_1) & set(days_in_range_2)
                    for day in common_days:
                        if (day, activity_id_1, activity_id_2) in maintenance_activities_seq_vars[section_id] or (day,
                                                                                                                  activity_id_2,
                                                                                                                  activity_id_1) in \
                                maintenance_activities_seq_vars[section_id]:
                            continue
                        if activity_id_1 < activity_id_2:
                            maintenance_activities_seq_vars[section_id][
                                (day, activity_id_1, activity_id_2)] = maintenance_planning_subproblem_models[
                                section_id].addVar(
                                vtype=GRB.BINARY,
                                name=f'maintenance_activities_seq_var_section_{section_id}_day_{day}_activities_{activity_id_1}_{activity_id_2}'
                            )
                        else:
                            maintenance_activities_seq_vars[section_id][
                                (day, activity_id_2, activity_id_1)] = maintenance_planning_subproblem_models[
                                section_id].addVar(
                                vtype=GRB.BINARY,
                                name=f'maintenance_activities_seq_var_section_{section_id}_day_{day}_activities_{activity_id_2}_{activity_id_1}'
                            )

In [ ]:
maintenance_activities_scheduling_simultaneously_indicator_vars = {}  # Variable indicating whether maintenance activities are scheduled simultaneously
maintenance_activities_start_end_time_seq_vars = {}  # Sequence variable for the start and end time of two maintenance activities

for section_id, maintenance_activities in section_maintenance_activities.items():
    maintenance_activities_scheduling_simultaneously_indicator_vars[section_id] = {}
    maintenance_activities_start_end_time_seq_vars[section_id] = {}

    for activity_id_1 in maintenance_activities:
        activity_type_1 = maintenance_activities_info[activity_id_1][0]
        # If activity_type_1 is in the mutually exclusive activity relationship dictionary, only activities that are not mutually exclusive with it are considered
        if activity_type_1 in mutually_exclusive_activities:
            for activity_id_2 in maintenance_activities:
                if activity_id_2 == activity_id_1:
                    continue
                activity_type_2 = maintenance_activities_info[activity_id_2][0]
                # If activity_type_2 and activity_type_1 are not mutually exclusive
                if activity_type_2 not in mutually_exclusive_activities[activity_type_1]:
                    start_date_1 = maintenance_activities_info[activity_id_1][1]
                    end_date_1 = maintenance_activities_info[activity_id_1][2]
                    days_in_range_1 = list(range(start_date_1, end_date_1 + 1))
                    start_date_2 = maintenance_activities_info[activity_id_2][1]
                    end_date_2 = maintenance_activities_info[activity_id_2][2]
                    days_in_range_2 = list(range(start_date_2, end_date_2 + 1))
                    # Take the intersection of two day lists to get the common day range
                    common_days = set(days_in_range_1) & set(days_in_range_2)
                    for day in common_days:
                        if (day, activity_id_1,
                            activity_id_2) not in maintenance_activities_scheduling_simultaneously_indicator_vars[
                            section_id] and (
                                day,
                                activity_id_2,
                                activity_id_1) not in maintenance_activities_scheduling_simultaneously_indicator_vars[
                            section_id]:
                            if activity_id_1 < activity_id_2:
                                maintenance_activities_scheduling_simultaneously_indicator_vars[section_id][
                                    (day, activity_id_1, activity_id_2)] = maintenance_planning_subproblem_models[
                                    section_id].addVar(
                                    vtype=GRB.BINARY,
                                    name=f'maintenance_activities_scheduling_simultaneously_indicator_var_section_{section_id}_day_{day}_activities_{activity_id_1}_{activity_id_2}'
                                )
                            else:
                                maintenance_activities_scheduling_simultaneously_indicator_vars[section_id][
                                    (day, activity_id_2, activity_id_1)] = maintenance_planning_subproblem_models[
                                    section_id].addVar(
                                    vtype=GRB.BINARY,
                                    name=f'maintenance_activities_scheduling_simultaneously_indicator_var_section_{section_id}_day_{day}_activities_{activity_id_2}_{activity_id_1}'
                                )
                        if (day, activity_id_1, activity_id_2) not in maintenance_activities_start_end_time_seq_vars[
                            section_id] and (
                                day, activity_id_2, activity_id_1) not in \
                                maintenance_activities_start_end_time_seq_vars[section_id]:
                            maintenance_activities_start_end_time_seq_vars[section_id][
                                (day, activity_id_1, activity_id_2)] = maintenance_planning_subproblem_models[
                                section_id].addVar(
                                vtype=GRB.BINARY,
                                name=f'maintenance_activities_start_end_time_seq_var_section_{section_id}_day_{day}_activities_{activity_id_1}_{activity_id_2}'
                            )
                            maintenance_activities_start_end_time_seq_vars[section_id][
                                (day, activity_id_2, activity_id_1)] = maintenance_planning_subproblem_models[
                                section_id].addVar(
                                vtype=GRB.BINARY,
                                name=f'maintenance_activities_start_end_time_seq_var_section_{section_id}_day_{day}_activities_{activity_id_2}_{activity_id_1}'
                            )
        # Otherwise, activity_id_1 can be paired with any other maintenance activity that is not mutually exclusive with it.
        else:
            for activity_id_2 in maintenance_activities:
                if activity_id_2 == activity_id_1:
                    continue
                activity_type_2 = maintenance_activities_info[activity_id_2][0]
                # If activity_type_2 is in the mutually exclusive activity relationship dictionary, check whether it is mutually exclusive with activity_type_1
                if activity_type_2 in mutually_exclusive_activities:
                    # If activity_type_2 is mutually exclusive with activity_type_1
                    if activity_type_1 in mutually_exclusive_activities[activity_type_2]:
                        continue
                start_date_1 = maintenance_activities_info[activity_id_1][1]
                end_date_1 = maintenance_activities_info[activity_id_1][2]
                days_in_range_1 = list(range(start_date_1, end_date_1 + 1))
                start_date_2 = maintenance_activities_info[activity_id_2][1]
                end_date_2 = maintenance_activities_info[activity_id_2][2]
                days_in_range_2 = list(range(start_date_2, end_date_2 + 1))
                # Take the intersection of two day lists to get the common day range
                common_days = set(days_in_range_1) & set(days_in_range_2)
                for day in common_days:
                    if (day, activity_id_1,
                        activity_id_2) not in maintenance_activities_scheduling_simultaneously_indicator_vars[
                        section_id] and (day,
                                         activity_id_2,
                                         activity_id_1) not in \
                            maintenance_activities_scheduling_simultaneously_indicator_vars[section_id]:
                        if activity_id_1 < activity_id_2:
                            maintenance_activities_scheduling_simultaneously_indicator_vars[section_id][
                                (day, activity_id_1, activity_id_2)] = maintenance_planning_subproblem_models[
                                section_id].addVar(
                                vtype=GRB.BINARY,
                                name=f'maintenance_activities_scheduling_simultaneously_indicator_var_section_{section_id}_day_{day}_activities_{activity_id_1}_{activity_id_2}'
                            )
                        else:
                            maintenance_activities_scheduling_simultaneously_indicator_vars[section_id][
                                (day, activity_id_2, activity_id_1)] = maintenance_planning_subproblem_models[
                                section_id].addVar(
                                vtype=GRB.BINARY,
                                name=f'maintenance_activities_scheduling_simultaneously_indicator_var_section_{section_id}_day_{day}_activities_{activity_id_2}_{activity_id_1}'
                            )
                    if (day, activity_id_1, activity_id_2) not in maintenance_activities_start_end_time_seq_vars[
                        section_id] and (
                            day, activity_id_2, activity_id_1) not in maintenance_activities_start_end_time_seq_vars[
                        section_id]:
                        maintenance_activities_start_end_time_seq_vars[section_id][
                            (day, activity_id_1, activity_id_2)] = maintenance_planning_subproblem_models[
                            section_id].addVar(
                            vtype=GRB.BINARY,
                            name=f'maintenance_activities_start_end_time_seq_var_section_{section_id}_day_{day}_activities_{activity_id_1}_{activity_id_2}'
                        )
                        maintenance_activities_start_end_time_seq_vars[section_id][
                            (day, activity_id_2, activity_id_1)] = maintenance_planning_subproblem_models[
                            section_id].addVar(
                            vtype=GRB.BINARY,
                            name=f'maintenance_activities_start_end_time_seq_var_section_{section_id}_day_{day}_activities_{activity_id_2}_{activity_id_1}'
                        )

#### Construct constraints

In [ ]:
maintenance_activity_day_scheduling_range_cons = {}  # Maintenance activity scheduling time range constraints
maintenance_activity_day_duration_cons = {}  # Maintenance activity duration constraints

for section_id, maintenance_activities in section_maintenance_activities.items():
    maintenance_activity_day_scheduling_range_cons[section_id] = {}
    maintenance_activity_day_duration_cons[section_id] = {}

    for day in range(1, len(num_of_trains) + 1):
        for activity_id in maintenance_activities:
            maintenance_activity_duration = maintenance_activities_info[activity_id][4]
            start_time_var = maintenance_activity_start_time_vars[section_id][(day, activity_id)]
            end_time_var = maintenance_activity_end_time_vars[section_id][(day, activity_id)]
            x_var = maintenance_activity_scheduling_day_vars[section_id][(day, activity_id)]

            # If the maintenance activity is not scheduled on that day, the start time and end time must be 0, otherwise the scheduling range is equivalent to the maintenance window scheduling range
            maintenance_activity_day_scheduling_range_cons[section_id][
                (day, activity_id, 1, 1)] = maintenance_planning_subproblem_models[section_id].addConstr(
                start_time_var <= MOT_l * x_var,
                name=f'maintenance_activity_start_range_con_section_{section_id}_day_{day}_activity_{activity_id}_1'
            )
            maintenance_activity_day_scheduling_range_cons[section_id][
                (day, activity_id, 1, 2)] = maintenance_planning_subproblem_models[section_id].addConstr(
                start_time_var >= MOT_e * x_var,
                name=f'maintenance_activity_start_range_con_section_{section_id}_day_{day}_activity_{activity_id}_2'
            )
            maintenance_activity_day_scheduling_range_cons[section_id][
                (day, activity_id, 2, 1)] = maintenance_planning_subproblem_models[section_id].addConstr(
                end_time_var <= MOT_l * x_var,
                name=f'maintenance_activity_end_range_con_section_{section_id}_day_{day}_activity_{activity_id}_1'
            )
            maintenance_activity_day_scheduling_range_cons[section_id][
                (day, activity_id, 2, 2)] = maintenance_planning_subproblem_models[section_id].addConstr(
                end_time_var >= MOT_e * x_var,
                name=f'maintenance_activity_end_range_con_section_{section_id}_day_{day}_activity_{activity_id}_2'
            )

            # If the maintenance activity is scheduled on that day, the duration must be equal to the expected value, otherwise it is 0
            maintenance_activity_day_duration_cons[section_id][
                (day, activity_id)] = maintenance_planning_subproblem_models[section_id].addConstr(
                end_time_var - start_time_var == maintenance_activity_duration * x_var,
                name=f'maintenance_activity_duration_con_section_{section_id}_day_{day}_activity_{activity_id}'
            )

In [ ]:
maintenance_activities_in_same_day_cons = {}  # Constraints related to whether the maintenance activities are within the same maintenance window

for section_id in maintenance_activities_in_same_day_indicator_vars.keys():
    maintenance_activities_in_same_day_cons[section_id] = {}

    for (day, activity_id_1, activity_id_2), var in maintenance_activities_in_same_day_indicator_vars[
        section_id].items():
        x_var_1 = maintenance_activity_scheduling_day_vars[section_id][(day, activity_id_1)]
        x_var_2 = maintenance_activity_scheduling_day_vars[section_id][(day, activity_id_2)]
        maintenance_activities_in_same_day_cons[section_id][
            (day, activity_id_1, activity_id_2, 1)] = maintenance_planning_subproblem_models[section_id].addConstr(
            var <= x_var_1,
            name=f'maintenance_activities_in_same_day_con_section_{section_id}_day_{day}_activities_{activity_id_1}_{activity_id_2}_1'
        )
        maintenance_activities_in_same_day_cons[section_id][
            (day, activity_id_1, activity_id_2, 2)] = maintenance_planning_subproblem_models[section_id].addConstr(
            var <= x_var_2,
            name=f'maintenance_activities_in_same_day_con_section_{section_id}_day_{day}_activities_{activity_id_1}_{activity_id_2}_2'
        )
        maintenance_activities_in_same_day_cons[section_id][
            (day, activity_id_1, activity_id_2, 3)] = maintenance_planning_subproblem_models[section_id].addConstr(
            var >= x_var_1 + x_var_2 - 1,
            name=f'maintenance_activities_in_same_day_con_section_{section_id}_day_{day}_activities_{activity_id_1}_{activity_id_2}_3'
        )

In [ ]:
maintenance_activities_seq_cons = {}  # Mutually exclusive maintenance activity scheduling sequence constraints

for section_id in maintenance_activities_seq_vars.keys():
    maintenance_activities_seq_cons[section_id] = {}

    for (day, activity_id_1, activity_id_2), var in maintenance_activities_seq_vars[section_id].items():
        start_time_var_1 = maintenance_activity_start_time_vars[section_id][(day, activity_id_1)]
        start_time_var_2 = maintenance_activity_start_time_vars[section_id][(day, activity_id_2)]
        end_time_var_1 = maintenance_activity_end_time_vars[section_id][(day, activity_id_1)]
        end_time_var_2 = maintenance_activity_end_time_vars[section_id][(day, activity_id_2)]

        # If activity_id_1 is before activity_id_2, the end time of activity_id_1 must be less than or equal to the start time of activity_id_2.
        maintenance_activities_seq_cons[section_id][
            (day, activity_id_1, activity_id_2, 1)] = maintenance_planning_subproblem_models[section_id].addConstr(
            start_time_var_2 + M * (1 - var) >= end_time_var_1,
            name=f'maintenance_activities_seq_con_section_{section_id}_day_{day}_activities_{activity_id_1}_{activity_id_2}_1'
        )

        # If activity_id_2 is before activity_id_1, the end time of activity_id_2 must be less than or equal to the start time of activity_id_1.
        maintenance_activities_seq_cons[section_id][
            (day, activity_id_1, activity_id_2, 2)] = maintenance_planning_subproblem_models[section_id].addConstr(
            start_time_var_1 + M * var >= end_time_var_2,
            name=f'maintenance_activities_seq_con_section_{section_id}_day_{day}_activities_{activity_id_1}_{activity_id_2}_2'
        )

In [ ]:
major_maintenance_activity_must_be_scheduled_cons = {}  # Constraints that major maintenance activities must be scheduled

for section_id, maintenance_activities in section_maintenance_activities.items():
    major_maintenance_activity_must_be_scheduled_cons[section_id] = {}

    for activity_id in maintenance_activities:
        activity_type = maintenance_activities_info[activity_id][0]
        # If it is a major maintenance activity
        if activity_type == 3 or activity_type == 4 or activity_type == 8:  # Major maintenance activity types are 3, 4 or 8
            start_date = maintenance_activities_info[activity_id][1]
            end_date = maintenance_activities_info[activity_id][2]
            for day in range(start_date, end_date + 1):
                major_maintenance_activity_must_be_scheduled_cons[section_id][
                    (day, activity_id)] = maintenance_planning_subproblem_models[section_id].addConstr(
                    maintenance_activity_scheduling_day_vars[section_id][(day, activity_id)] == 1,
                    name=f'major_maintenance_activity_must_be_scheduled_con_section_{section_id}_day_{day}_activity_{activity_id}'
                )

In [ ]:
routine_maintenance_activity_most_once_scheduled_cons = {}  # Constraints that routine maintenance activities are scheduled on only one day

for section_id, maintenance_activities in section_maintenance_activities.items():
    routine_maintenance_activity_most_once_scheduled_cons[section_id] = {}

    for activity_id in maintenance_activities:
        activity_type = maintenance_activities_info[activity_id][0]
        # If it is a routine maintenance activity
        if activity_type != 3 and activity_type != 4 and activity_type != 8:
            start_date = maintenance_activities_info[activity_id][1]
            end_date = maintenance_activities_info[activity_id][2]
            temp_str = gp.LinExpr(0)
            for day in range(start_date, end_date + 1):
                temp_str += maintenance_activity_scheduling_day_vars[section_id][(day, activity_id)]
            routine_maintenance_activity_most_once_scheduled_cons[section_id][
                activity_id] = maintenance_planning_subproblem_models[section_id].addConstr(
                temp_str <= 1,
                name=f'routine_maintenance_activity_most_once_scheduled_con_section_{section_id}_activity_{activity_id}'
            )

#### Construct objective function

At each iteration, the objective function needs to be updated according to the updated Lagrangian multipliers

In [ ]:
def define_or_update_obj_of_maintenance_planning_subproblem_models():
    for inner_section_id, inner_maintenance_activities in section_maintenance_activities.items():

        obj_of_maintenance_planning_subproblem_model = gp.LinExpr(0)
        for inner_activity_id in inner_maintenance_activities:
            inner_activity_type = maintenance_activities_info[inner_activity_id][0]
            if inner_activity_type == 3 or inner_activity_type == 4 or inner_activity_type == 8:  # 大修作业类型为3、4或8
                continue
            inner_start_date = maintenance_activities_info[inner_activity_id][1]
            inner_end_date = maintenance_activities_info[inner_activity_id][2]
            inner_importance_level = maintenance_activities_info[inner_activity_id][5]
            inner_temp_str = gp.LinExpr(0)
            for inner_day in range(inner_start_date, inner_end_date + 1):
                inner_x_var = maintenance_activity_scheduling_day_vars[inner_section_id][(inner_day, inner_activity_id)]
                inner_temp_str += inner_x_var
            obj_of_maintenance_planning_subproblem_model += w2 * importance_penalty_costs[inner_importance_level] * (
                    1 - inner_temp_str)
        for inner_day in range(1, len(num_of_trains) + 1):
            for inner_activity_id in inner_maintenance_activities:
                inner_lagrangian_multiplier_of_start = \
                    lagrangian_multipliers[inner_day][inner_section_id][inner_activity_id][1]
                inner_lagrangian_multiplier_of_end = \
                    lagrangian_multipliers[inner_day][inner_section_id][inner_activity_id][2]
                inner_x_var = maintenance_activity_scheduling_day_vars[inner_section_id][(inner_day, inner_activity_id)]
                inner_start_time_var = maintenance_activity_start_time_vars[inner_section_id][
                    (inner_day, inner_activity_id)]
                inner_end_time_var = maintenance_activity_end_time_vars[inner_section_id][
                    (inner_day, inner_activity_id)]
                obj_of_maintenance_planning_subproblem_model += inner_lagrangian_multiplier_of_start * (
                        M * inner_x_var - inner_start_time_var)
                obj_of_maintenance_planning_subproblem_model += inner_lagrangian_multiplier_of_end * (
                        M * inner_x_var + inner_end_time_var)
        maintenance_planning_subproblem_models[inner_section_id].setObjective(
            obj_of_maintenance_planning_subproblem_model,
            GRB.MINIMIZE)  # The objective function is to minimize
        maintenance_planning_subproblem_models[inner_section_id].update()  # Update model

## Defining the key components of the Lagrangian relaxation-based heuristic algorithm

Define the functions to obtain the lower bound solution by solving the train timetabling with maintenance window setting subproblem of each day and the maintenance scheduling subproblem of each section.

In [ ]:
# Obtain the lower bound solution of the daily train timetabling with maintenance window setting subproblem
def get_lower_bound_solution_of_daily_train_timetables_with_maintenance_windows_subproblem(inner_day):
    # Define the callback function related to constraints (17)-(20)
    def station_track_capacity_con_lazy_callback(inner_model, where):
        global violated_station_track_capacity_con_count  # Station capacity limit constraint violation count
        if where == GRB.Callback.MIPSOL:
            # Get the arrival and departure time of trains at each station, and the start and end time of maintenance windows in the current feasible solution
            inner_dep_time_sol_of_actual_trains = {name: int(inner_model.cbGetSolution(inner_dep_time_var)) for
                                                   name, inner_dep_time_var in
                                                   inner_model._dep_time_vars_of_actual_trains.items()}
            inner_arr_time_sol_of_actual_trains = {name: int(inner_model.cbGetSolution(inner_arr_time_var)) for
                                                   name, inner_arr_time_var in
                                                   inner_model._arr_time_vars_of_actual_trains.items()}

            # Judge whether any station capacity-related constraint is violated. For each intermediate station, traverse each train in turn, obtain its arrival time and departure time, then traverse other trains and count the number of trains that stop at the station at the same time. If the station capacity limit is exceeded, first record the station ID and the IDs of these trains, and finally add the relevant constraints to the station capacity-related constraint pool.
            violated_station_track_capacity_con_trains = {}
            daily_train_info = train_info[inner_day]
            for inner_station_id in range(2, num_of_stations):
                violated_station_track_capacity_con_trains[inner_station_id] = []
                train_id_list = sorted(daily_train_info.keys())
                for inner_train_id_1 in train_id_list:
                    train_count_at_station = 0
                    inner_arr_time_1 = inner_arr_time_sol_of_actual_trains[(inner_train_id_1, inner_station_id)]
                    inner_dep_time_1 = inner_dep_time_sol_of_actual_trains[(inner_train_id_1, inner_station_id)]
                    if abs(inner_dep_time_1 - inner_arr_time_1) < 1e-6:  # If the departure time and arrival time are equal, the train is considered to pass the station and can be skipped for statistics.
                        continue
                    else:
                        train_count_at_station += 1
                        meeting_trains = [inner_train_id_1]
                        for inner_train_id_2 in train_id_list:
                            if inner_train_id_2 > inner_train_id_1:
                                inner_arr_time_2 = inner_arr_time_sol_of_actual_trains[
                                    (inner_train_id_2, inner_station_id)]
                                inner_dep_time_2 = inner_dep_time_sol_of_actual_trains[
                                    (inner_train_id_2, inner_station_id)]
                                if abs(inner_dep_time_2 - inner_arr_time_2) < 1e-6: continue
                                # If the departure time of train 2 is after the arrival time of train 1, and the arrival time of train 2 is before the departure time of train 1, then they are considered to meet at the station
                                if inner_dep_time_2 > inner_arr_time_1 and inner_arr_time_2 < inner_dep_time_1:
                                    train_count_at_station += 1
                                    meeting_trains.append(inner_train_id_2)
                        # Check whether the length of the list 'meeting_trains' exceeds the station capacity limit
                        if train_count_at_station > station_track_capacity or len(
                                meeting_trains) > station_track_capacity:
                            violated_station_track_capacity_con_trains[inner_station_id].append(
                                sorted(meeting_trains))

            station_capacity_limitation_cons = {}  # Create the station capacity-related constraint pool to avoid repeated addition of constraints. The key is the station ID and the value is the constraint dictionary.
            # For each train set that violates the station capacity limit at each station, a subset of all 'station_track_capacity' train IDs is taken from the set without duplication, the corresponding constraints are constructed, and stored in 'station_capacity_limitation_cons'. By managing key-value pairs, constraints can be added without duplication.
            for inner_station_id, inner_train_lists in violated_station_track_capacity_con_trains.items():
                station_capacity_limitation_cons[inner_station_id] = {}
                for inner_train_ids in inner_train_lists:
                    for inner_train_combination in combinations(inner_train_ids, station_track_capacity + 1):
                        inner_train_combination = sorted(inner_train_combination)
                        for inner_train_id_1 in inner_train_combination:
                            for inner_train_id_2 in inner_train_combination:
                                if inner_train_id_2 > inner_train_id_1:
                                    if (inner_train_id_1, inner_train_id_2, 1) in station_capacity_limitation_cons[
                                        inner_station_id] or (inner_train_id_1, inner_train_id_2, 2) in \
                                            station_capacity_limitation_cons[inner_station_id] or (inner_train_id_1,
                                                                                                   inner_train_id_2,
                                                                                                   3) in \
                                            station_capacity_limitation_cons[inner_station_id] or (inner_train_id_1,
                                                                                                   inner_train_id_2,
                                                                                                   4) in \
                                            station_capacity_limitation_cons[inner_station_id] or (inner_train_id_1,
                                                                                                   inner_train_id_2,
                                                                                                   5) in \
                                            station_capacity_limitation_cons[inner_station_id]:
                                        continue
                                    station_capacity_limitation_cons[inner_station_id][
                                        (inner_train_id_1, inner_train_id_2, 1)] = inner_model.cbLazy(
                                        M * (inner_model._train_arr_dep_seq_vars[
                                                 (inner_train_id_1, inner_train_id_2, inner_station_id)] - 1) <=
                                        inner_model._dep_time_vars_of_actual_trains[
                                            (inner_train_id_2, inner_station_id)] -
                                        inner_model._arr_time_vars_of_actual_trains[
                                            (inner_train_id_1, inner_station_id)]
                                    )
                                    station_capacity_limitation_cons[inner_station_id][
                                        (inner_train_id_1, inner_train_id_2, 2)] = inner_model.cbLazy(
                                        inner_model._dep_time_vars_of_actual_trains[
                                            (inner_train_id_2, inner_station_id)] -
                                        inner_model._arr_time_vars_of_actual_trains[
                                            (inner_train_id_1, inner_station_id)] <= M *
                                        inner_model._train_arr_dep_seq_vars[
                                            (inner_train_id_1, inner_train_id_2, inner_station_id)] - hda
                                    )
                                    station_capacity_limitation_cons[inner_station_id][
                                        (inner_train_id_1, inner_train_id_2, 3)] = inner_model.cbLazy(
                                        M * (inner_model._train_arr_dep_seq_vars[
                                                 (inner_train_id_2, inner_train_id_1, inner_station_id)] - 1) <=
                                        inner_model._dep_time_vars_of_actual_trains[
                                            (inner_train_id_1, inner_station_id)] -
                                        inner_model._arr_time_vars_of_actual_trains[
                                            (inner_train_id_2, inner_station_id)]
                                    )
                                    station_capacity_limitation_cons[inner_station_id][
                                        (inner_train_id_1, inner_train_id_2, 4)] = inner_model.cbLazy(
                                        inner_model._dep_time_vars_of_actual_trains[
                                            (inner_train_id_1, inner_station_id)] -
                                        inner_model._arr_time_vars_of_actual_trains[
                                            (inner_train_id_2, inner_station_id)] <= M *
                                        inner_model._train_arr_dep_seq_vars[
                                            (inner_train_id_2, inner_train_id_1, inner_station_id)] - hda
                                    )
                                    station_capacity_limitation_cons[inner_station_id][
                                        (inner_train_id_1, inner_train_id_2, 5)] = inner_model.cbLazy(
                                        inner_model._train_meeting_vars[
                                            (inner_train_id_1, inner_train_id_2, inner_station_id)] ==
                                        inner_model._train_arr_dep_seq_vars[
                                            (inner_train_id_1, inner_train_id_2, inner_station_id)] +
                                        inner_model._train_arr_dep_seq_vars[
                                            (inner_train_id_2, inner_train_id_1, inner_station_id)] - 1
                                    )
                                    violated_station_track_capacity_con_count += 5
                        if tuple(inner_train_combination) not in station_capacity_limitation_cons[inner_station_id]:
                            temp_expr = gp.LinExpr(0)
                            for inner_train_id_1 in inner_train_combination:
                                for inner_train_id_2 in inner_train_combination:
                                    if inner_train_id_2 > inner_train_id_1:
                                        temp_expr += inner_model._train_meeting_vars[
                                            (inner_train_id_1, inner_train_id_2, inner_station_id)]
                            station_capacity_limitation_cons[inner_station_id][
                                tuple(inner_train_combination)] = inner_model.cbLazy(
                                temp_expr <= math.comb(station_track_capacity + 1, 2) - 1
                            )
                            violated_station_track_capacity_con_count += 1

    inner_daily_model = train_timetables_with_maintenance_windows_subproblem_models[inner_day]
    # Save variable references to model object for callback access
    inner_daily_model._dep_time_vars_of_actual_trains = dep_time_vars_of_actual_trains[inner_day]
    inner_daily_model._arr_time_vars_of_actual_trains = arr_time_vars_of_actual_trains[inner_day]
    inner_daily_model._train_arr_dep_seq_vars = train_arr_dep_seq_vars[inner_day]
    inner_daily_model._train_meeting_vars = train_meeting_vars[inner_day]

    # Set the solution parameters
    inner_daily_model.setParam('OutputFlag', 0)  # Turn off Gurobi logging
    inner_daily_model.Params.lazyConstraints = 1
    inner_daily_model.setParam('TimeLimit', 120)  # Set the solution time limit to 2 min
    inner_daily_model.setParam('MIPGap', 0.015)
    inner_daily_model.optimize(station_track_capacity_con_lazy_callback)

In [ ]:
# Obtain the lower bound solution of the section-specific subproblem related to maintenance scheduling within the planning horizon
def get_lower_bound_solution_of_maintenance_planning_subproblem(inner_section_id):
    # Define the callback function related to constraints (31)-(35)
    def maintenance_activity_scheduling_simultaneously_lazy_callback(inner_model, where):
        global violated_maintenance_activity_scheduling_simultaneously_con_count  # Count of the violations of the constraints related to the limited number of compatible maintenance activities scheduled simultaneously
        if where == GRB.Callback.MIPSOL:
            # Get the maintenance activity start and end time in the current feasible solution
            inner_maintenance_activity_start_time_sol = {name: int(inner_model.cbGetSolution(inner_start_time_var)) for
                                                         name, inner_start_time_var in
                                                         inner_model._maintenance_activity_start_time_vars.items()}
            inner_maintenance_activity_end_time_sol = {name: int(inner_model.cbGetSolution(inner_end_time_var)) for
                                                       name, inner_end_time_var in
                                                       inner_model._maintenance_activity_end_time_vars.items()}
            inner_maintenance_activity_scheduling_day_sol = {name: int(inner_model.cbGetSolution(inner_x_var)) for
                                                             name, inner_x_var in
                                                             inner_model._maintenance_activity_scheduling_day_vars.items()}

            # Judge whether the limit on the number of maintenance activities scheduled simultaneously is violated. For each section, traverse each maintenance activity in turn, obtain its start time and end time, then traverse other maintenance activities, and count the number of maintenance activities scheduled simultaneously with it. If the limit on the number of maintenance activities scheduled simultaneously in the section is exceeded, first record the section ID and the IDs of these maintenance activities, and finally add the relevant constraints to the maintenance activity simultaneous scheduling limit constraint pool.
            violated_maintenance_activity_simultaneous_number_con_maintenance_activities = {}
            for inner_day in range(1, len(num_of_trains) + 1):
                violated_maintenance_activity_simultaneous_number_con_maintenance_activities[
                    inner_day] = []
                inner_maintenance_activities = sorted(section_maintenance_activities[inner_section_id])
                for inner_activity_id_1 in inner_maintenance_activities:
                    simultaneous_count = 0
                    if inner_maintenance_activity_scheduling_day_sol[
                        (inner_day, inner_activity_id_1)] == 0:
                        continue
                    # Get the start and end time of the maintenance activity
                    inner_maintenance_activity_start_time_1 = inner_maintenance_activity_start_time_sol[
                        (inner_day, inner_activity_id_1)]
                    inner_maintenance_activity_end_time_1 = inner_maintenance_activity_end_time_sol[
                        (inner_day, inner_activity_id_1)]
                    simultaneous_count += 1
                    inner_meeting_activities = [inner_activity_id_1]
                    for inner_activity_id_2 in inner_maintenance_activities:
                        if inner_activity_id_2 > inner_activity_id_1:
                            if inner_maintenance_activity_scheduling_day_sol[
                                (inner_day, inner_activity_id_2)] == 0:
                                continue
                            inner_maintenance_activity_start_time_2 = inner_maintenance_activity_start_time_sol[
                                (inner_day, inner_activity_id_2)]
                            inner_maintenance_activity_end_time_2 = inner_maintenance_activity_end_time_sol[
                                (inner_day, inner_activity_id_2)]
                            # If two maintenance activities start before each other's end time, they are considered to be scheduled simultaneously.
                            if inner_maintenance_activity_end_time_2 > inner_maintenance_activity_start_time_1 and inner_maintenance_activity_end_time_1 > inner_maintenance_activity_start_time_2:
                                simultaneous_count += 1
                                inner_meeting_activities.append(inner_activity_id_2)
                    if simultaneous_count > max_compatible_activities or len(
                            inner_meeting_activities) > max_compatible_activities:
                        violated_maintenance_activity_simultaneous_number_con_maintenance_activities[
                            inner_day].append(sorted(
                            inner_meeting_activities))

            maintenance_activity_simultaneous_number_limitation_cons = {}  # Create a constraint pool to avoid adding repeated constraints. The outer key is day, the inner key is section ID, and the value is constraint dictionary
            # For each maintenance activity set that violates the maintenance activity simultaneous scheduling limit stored in each section, extract a subset of all 'max_compatible_activities' maintenance activity IDs from the set without duplication, construct the corresponding constraint, and store it in 'maintenance_activity_simultaneous_number_limitation_cons'. Through key-value pair management, constraints can be added without duplication.
            for inner_day, inner_violated_maintenance_activities in violated_maintenance_activity_simultaneous_number_con_maintenance_activities.items():
                maintenance_activity_simultaneous_number_limitation_cons[inner_day] = {}
                for inner_activity_list in inner_violated_maintenance_activities:
                    for inner_activity_combination in combinations(inner_activity_list, max_compatible_activities + 1):
                        inner_activity_combination = sorted(inner_activity_combination)
                        for inner_activity_id_1 in inner_activity_combination:
                            for inner_activity_id_2 in inner_activity_combination:
                                if inner_activity_id_2 > inner_activity_id_1:
                                    if (inner_activity_id_1, inner_activity_id_2, 1) in \
                                            maintenance_activity_simultaneous_number_limitation_cons[inner_day] or (
                                            inner_activity_id_1, inner_activity_id_2,
                                            2) in \
                                            maintenance_activity_simultaneous_number_limitation_cons[inner_day] or (
                                            inner_activity_id_1, inner_activity_id_2,
                                            3) in \
                                            maintenance_activity_simultaneous_number_limitation_cons[inner_day] or (
                                            inner_activity_id_1, inner_activity_id_2,
                                            4) in \
                                            maintenance_activity_simultaneous_number_limitation_cons[inner_day] or (
                                            inner_activity_id_1, inner_activity_id_2,
                                            5) in \
                                            maintenance_activity_simultaneous_number_limitation_cons[inner_day] or (
                                            inner_activity_id_1, inner_activity_id_2,
                                            6) in \
                                            maintenance_activity_simultaneous_number_limitation_cons[inner_day]:
                                        continue
                                    if (inner_day, inner_activity_id_1,
                                        inner_activity_id_2) in inner_model._maintenance_activities_start_end_time_seq_vars and (
                                            inner_day, inner_activity_id_2,
                                            inner_activity_id_1) in inner_model._maintenance_activities_start_end_time_seq_vars:
                                        maintenance_activity_simultaneous_number_limitation_cons[inner_day][
                                            (inner_activity_id_1, inner_activity_id_2, 1)] = inner_model.cbLazy(
                                            M * (inner_model._maintenance_activities_start_end_time_seq_vars[
                                                     (inner_day, inner_activity_id_1, inner_activity_id_2)] - 1) <=
                                            inner_model._maintenance_activity_end_time_vars[
                                                (inner_day, inner_activity_id_2)] -
                                            inner_model._maintenance_activity_start_time_vars[
                                                (inner_day, inner_activity_id_1)]
                                        )
                                        maintenance_activity_simultaneous_number_limitation_cons[inner_day][
                                            (inner_activity_id_1, inner_activity_id_2, 2)] = inner_model.cbLazy(
                                            inner_model._maintenance_activity_end_time_vars[
                                                (inner_day, inner_activity_id_2)] -
                                            inner_model._maintenance_activity_start_time_vars[
                                                (inner_day, inner_activity_id_1)] <= M *
                                            inner_model._maintenance_activities_start_end_time_seq_vars[
                                                (inner_day, inner_activity_id_1, inner_activity_id_2)]
                                        )
                                        maintenance_activity_simultaneous_number_limitation_cons[inner_day][
                                            (inner_activity_id_1, inner_activity_id_2, 3)] = inner_model.cbLazy(
                                            M * (inner_model._maintenance_activities_start_end_time_seq_vars[
                                                     (inner_day, inner_activity_id_2, inner_activity_id_1)] - 1) <=
                                            inner_model._maintenance_activity_end_time_vars[
                                                (inner_day, inner_activity_id_1)] -
                                            inner_model._maintenance_activity_start_time_vars[
                                                (inner_day, inner_activity_id_2)]
                                        )
                                        maintenance_activity_simultaneous_number_limitation_cons[inner_day][
                                            (inner_activity_id_1, inner_activity_id_2, 4)] = inner_model.cbLazy(
                                            inner_model._maintenance_activity_end_time_vars[
                                                (inner_day, inner_activity_id_1)] -
                                            inner_model._maintenance_activity_start_time_vars[
                                                (inner_day, inner_activity_id_2)] <= M *
                                            inner_model._maintenance_activities_start_end_time_seq_vars[
                                                (inner_day, inner_activity_id_2, inner_activity_id_1)]
                                        )
                                        maintenance_activity_simultaneous_number_limitation_cons[inner_day][
                                            (inner_activity_id_1, inner_activity_id_2, 5)] = inner_model.cbLazy(
                                            inner_model._maintenance_activities_scheduling_simultaneously_indicator_vars[
                                                (inner_day, inner_activity_id_1, inner_activity_id_2)] ==
                                            inner_model._maintenance_activities_start_end_time_seq_vars[
                                                (inner_day, inner_activity_id_1, inner_activity_id_2)] +
                                            inner_model._maintenance_activities_start_end_time_seq_vars[
                                                (inner_day, inner_activity_id_2, inner_activity_id_1)] - 1
                                        )
                                        maintenance_activity_simultaneous_number_limitation_cons[inner_day][
                                            (inner_activity_id_1, inner_activity_id_2, 6)] = inner_model.cbLazy(
                                            inner_model._maintenance_activities_scheduling_simultaneously_indicator_vars[
                                                (inner_day, inner_activity_id_1, inner_activity_id_2)] <=
                                            inner_model._maintenance_activities_in_same_day_indicator_vars[
                                                (inner_day, inner_activity_id_1, inner_activity_id_2)]
                                        )
                                        violated_maintenance_activity_scheduling_simultaneously_con_count += 6
                        if tuple(inner_activity_combination) not in \
                                maintenance_activity_simultaneous_number_limitation_cons[inner_day]:
                            temp_expr = gp.LinExpr(0)
                            for inner_activity_id_1 in inner_activity_combination:
                                for inner_activity_id_2 in inner_activity_combination:
                                    if inner_activity_id_2 > inner_activity_id_1:
                                        if (inner_day, inner_activity_id_1,
                                            inner_activity_id_2) in inner_model._maintenance_activities_scheduling_simultaneously_indicator_vars:
                                            temp_expr += \
                                                inner_model._maintenance_activities_scheduling_simultaneously_indicator_vars[
                                                    (inner_day, inner_activity_id_1, inner_activity_id_2)]
                            maintenance_activity_simultaneous_number_limitation_cons[inner_day][
                                tuple(inner_activity_combination)] = inner_model.cbLazy(
                                temp_expr <= math.comb(max_compatible_activities + 1, 2) - 1
                            )
                            violated_maintenance_activity_scheduling_simultaneously_con_count += 1

    inner_section_maintenance_planning_model = maintenance_planning_subproblem_models[inner_section_id]
    # Save variable references to model object for callback access
    inner_section_maintenance_planning_model._maintenance_activity_start_time_vars = \
        maintenance_activity_start_time_vars[inner_section_id]
    inner_section_maintenance_planning_model._maintenance_activity_end_time_vars = maintenance_activity_end_time_vars[
        inner_section_id]
    inner_section_maintenance_planning_model._maintenance_activity_scheduling_day_vars = \
        maintenance_activity_scheduling_day_vars[inner_section_id]
    inner_section_maintenance_planning_model._maintenance_activities_start_end_time_seq_vars = \
        maintenance_activities_start_end_time_seq_vars[inner_section_id]
    inner_section_maintenance_planning_model._maintenance_activities_scheduling_simultaneously_indicator_vars = \
        maintenance_activities_scheduling_simultaneously_indicator_vars[inner_section_id]
    inner_section_maintenance_planning_model._maintenance_activities_in_same_day_indicator_vars = \
        maintenance_activities_in_same_day_indicator_vars[inner_section_id]

    # Set the solution parameters
    inner_section_maintenance_planning_model.setParam('OutputFlag', 0)
    inner_section_maintenance_planning_model.Params.lazyConstraints = 1
    inner_section_maintenance_planning_model.setParam('TimeLimit', 60)
    inner_section_maintenance_planning_model.setParam('MIPGap', 0.01)
    inner_section_maintenance_planning_model.optimize(maintenance_activity_scheduling_simultaneously_lazy_callback)

Define functions related to obtaining the upper bound solution, including obtaining a list of maintenance activities actually scheduled for each interval each day, and functions for determining whether unscheduled maintenance activities can be scheduled to another day.

By inputting the maintenance activity list obtained by solving the lower bound solution, we can obtain the maintenance activity list actually scheduled within the maintenance window of the corresponding section on that day.

In [ ]:
def obtain_actually_scheduled_maintenance_activities(input_maintenance_activity_ids,
                                                     input_maintenance_window_start_time,
                                                     input_maintenance_window_end_time):
    # Construct model
    obtain_actually_scheduled_maintenance_activities_model = gp.Model(
        "obtain_actually_scheduled_maintenance_activities_model")

    # Define variables, their meanings are similar to those introduced before.
    inner_maintenance_activity_scheduling_vars = {}
    inner_maintenance_activity_start_time_vars = {}
    inner_maintenance_activity_end_time_vars = {}
    for inner_activity_id in input_maintenance_activity_ids:
        inner_maintenance_activity_scheduling_vars[
            inner_activity_id] = obtain_actually_scheduled_maintenance_activities_model.addVar(
            vtype=GRB.BINARY,
            name=f'inner_maintenance_activity_scheduling_var_activity_{inner_activity_id}')
        inner_maintenance_activity_start_time_vars[
            inner_activity_id] = obtain_actually_scheduled_maintenance_activities_model.addVar(
            lb=0,
            ub=input_maintenance_window_end_time,
            vtype=GRB.INTEGER,
            name=f'inner_maintenance_activity_start_time_var_activity_{inner_activity_id}'
        )
        inner_maintenance_activity_end_time_vars[
            inner_activity_id] = obtain_actually_scheduled_maintenance_activities_model.addVar(
            lb=0,
            ub=input_maintenance_window_end_time,
            vtype=GRB.INTEGER,
            name=f'inner_maintenance_activity_end_time_var_activity_{inner_activity_id}'
        )

    inner_maintenance_activities_both_scheduling_indicator_vars = {}
    for inner_activity_id_1 in input_maintenance_activity_ids:
        for inner_activity_id_2 in input_maintenance_activity_ids:
            if inner_activity_id_1 < inner_activity_id_2:
                inner_maintenance_activities_both_scheduling_indicator_vars[(inner_activity_id_1,
                                                                             inner_activity_id_2)] = obtain_actually_scheduled_maintenance_activities_model.addVar(
                    vtype=GRB.BINARY,
                    name=f'inner_maintenance_activities_both_scheduling_indicator_var_activities_{inner_activity_id_1}_{inner_activity_id_2}'
                )

    inner_maintenance_activities_seq_vars = {}
    for inner_activity_id_1 in input_maintenance_activity_ids:
        inner_activity_type_1 = maintenance_activities_info[inner_activity_id_1][0]
        if inner_activity_type_1 in mutually_exclusive_activities:
            for inner_activity_id_2 in input_maintenance_activity_ids:
                if inner_activity_id_2 == inner_activity_id_1:
                    continue
                inner_activity_type_2 = maintenance_activities_info[inner_activity_id_2][0]
                if inner_activity_type_2 in mutually_exclusive_activities[inner_activity_type_1]:
                    if (inner_activity_id_1, inner_activity_id_2) in inner_maintenance_activities_seq_vars or (
                            inner_activity_id_2,
                            inner_activity_id_1) in inner_maintenance_activities_seq_vars:
                        continue
                    if inner_activity_id_1 < inner_activity_id_2:
                        inner_maintenance_activities_seq_vars[(inner_activity_id_1,
                                                               inner_activity_id_2)] = obtain_actually_scheduled_maintenance_activities_model.addVar(
                            vtype=GRB.BINARY,
                            name=f'inner_maintenance_activity_seq_var_activities_{inner_activity_id_1}_{inner_activity_id_2}'
                        )
                    else:
                        inner_maintenance_activities_seq_vars[(inner_activity_id_2,
                                                               inner_activity_id_1)] = obtain_actually_scheduled_maintenance_activities_model.addVar(
                            vtype=GRB.BINARY,
                            name=f'inner_maintenance_activity_seq_var_activities_{inner_activity_id_2}_{inner_activity_id_1}'
                        )

    inner_maintenance_activities_scheduling_simultaneously_indicator_vars = {}
    inner_maintenance_activities_start_end_time_seq_vars = {}
    for inner_activity_id_1 in input_maintenance_activity_ids:
        inner_activity_type_1 = maintenance_activities_info[inner_activity_id_1][0]
        if inner_activity_type_1 in mutually_exclusive_activities:
            for inner_activity_id_2 in input_maintenance_activity_ids:
                if inner_activity_id_2 == inner_activity_id_1:
                    continue
                inner_activity_type_2 = maintenance_activities_info[inner_activity_id_2][0]
                if inner_activity_type_2 not in mutually_exclusive_activities[inner_activity_type_1]:
                    if (inner_activity_id_1,
                        inner_activity_id_2) not in inner_maintenance_activities_scheduling_simultaneously_indicator_vars and (
                            inner_activity_id_2,
                            inner_activity_id_1) not in inner_maintenance_activities_scheduling_simultaneously_indicator_vars:
                        if inner_activity_id_1 < inner_activity_id_2:
                            inner_maintenance_activities_scheduling_simultaneously_indicator_vars[
                                (inner_activity_id_1,
                                 inner_activity_id_2)] = obtain_actually_scheduled_maintenance_activities_model.addVar(
                                vtype=GRB.BINARY,
                                name=f'inner_maintenance_activities_scheduling_simultaneously_indicator_var_activities_{inner_activity_id_1}_{inner_activity_id_2}'
                            )
                        else:
                            inner_maintenance_activities_scheduling_simultaneously_indicator_vars[
                                (inner_activity_id_2,
                                 inner_activity_id_1)] = obtain_actually_scheduled_maintenance_activities_model.addVar(
                                vtype=GRB.BINARY,
                                name=f'inner_maintenance_activities_scheduling_simultaneously_indicator_var_activities_{inner_activity_id_2}_{inner_activity_id_1}'
                            )
                    if (inner_activity_id_1,
                        inner_activity_id_2) not in inner_maintenance_activities_start_end_time_seq_vars and (
                            inner_activity_id_2,
                            inner_activity_id_1) not in inner_maintenance_activities_start_end_time_seq_vars:
                        inner_maintenance_activities_start_end_time_seq_vars[
                            (inner_activity_id_1,
                             inner_activity_id_2)] = obtain_actually_scheduled_maintenance_activities_model.addVar(
                            vtype=GRB.BINARY,
                            name=f'inner_maintenance_activities_start_end_time_seq_var_activities_{inner_activity_id_1}_{inner_activity_id_2}'
                        )
                        inner_maintenance_activities_start_end_time_seq_vars[
                            (inner_activity_id_2,
                             inner_activity_id_1)] = obtain_actually_scheduled_maintenance_activities_model.addVar(
                            vtype=GRB.BINARY,
                            name=f'inner_maintenance_activities_start_end_time_seq_var_activities_{inner_activity_id_2}_{inner_activity_id_1}'
                        )
        else:
            for inner_activity_id_2 in input_maintenance_activity_ids:
                if inner_activity_id_2 == inner_activity_id_1:
                    continue
                inner_activity_type_2 = maintenance_activities_info[inner_activity_id_2][0]
                if inner_activity_type_2 in mutually_exclusive_activities:
                    if inner_activity_type_1 in mutually_exclusive_activities[inner_activity_type_2]:
                        continue
                if (inner_activity_id_1,
                    inner_activity_id_2) not in inner_maintenance_activities_scheduling_simultaneously_indicator_vars and (
                        inner_activity_id_2,
                        inner_activity_id_1) not in inner_maintenance_activities_scheduling_simultaneously_indicator_vars:
                    if inner_activity_id_1 < inner_activity_id_2:
                        inner_maintenance_activities_scheduling_simultaneously_indicator_vars[
                            (inner_activity_id_1,
                             inner_activity_id_2)] = obtain_actually_scheduled_maintenance_activities_model.addVar(
                            vtype=GRB.BINARY,
                            name=f'inner_maintenance_activities_scheduling_simultaneously_indicator_var_activities_{inner_activity_id_1}_{inner_activity_id_2}'
                        )
                    else:
                        inner_maintenance_activities_scheduling_simultaneously_indicator_vars[
                            (inner_activity_id_2,
                             inner_activity_id_1)] = obtain_actually_scheduled_maintenance_activities_model.addVar(
                            vtype=GRB.BINARY,
                            name=f'inner_maintenance_activities_scheduling_simultaneously_indicator_var_activities_{inner_activity_id_2}_{inner_activity_id_1}'
                        )
                if (inner_activity_id_1,
                    inner_activity_id_2) not in inner_maintenance_activities_start_end_time_seq_vars and (
                        inner_activity_id_2,
                        inner_activity_id_1) not in inner_maintenance_activities_start_end_time_seq_vars:
                    inner_maintenance_activities_start_end_time_seq_vars[
                        (inner_activity_id_1,
                         inner_activity_id_2)] = obtain_actually_scheduled_maintenance_activities_model.addVar(
                        vtype=GRB.BINARY,
                        name=f'inner_maintenance_activities_start_end_time_seq_var_activities_{inner_activity_id_1}_{inner_activity_id_2}'
                    )
                    inner_maintenance_activities_start_end_time_seq_vars[
                        (inner_activity_id_2,
                         inner_activity_id_1)] = obtain_actually_scheduled_maintenance_activities_model.addVar(
                        vtype=GRB.BINARY,
                        name=f'inner_maintenance_activities_start_end_time_seq_var_activities_{inner_activity_id_2}_{inner_activity_id_1}'
                    )

    # Define constraints, their meanings are similar to those introduced before.
    inner_maintenance_activity_scheduling_range_cons = {}
    inner_maintenance_activity_duration_cons = {}

    for inner_activity_id in input_maintenance_activity_ids:
        inner_maintenance_activity_duration = maintenance_activities_info[inner_activity_id][4]
        inner_start_time_var = inner_maintenance_activity_start_time_vars[inner_activity_id]
        inner_end_time_var = inner_maintenance_activity_end_time_vars[inner_activity_id]
        inner_x_var = inner_maintenance_activity_scheduling_vars[inner_activity_id]

        inner_maintenance_activity_scheduling_range_cons[
            (inner_activity_id, 1, 1)] = obtain_actually_scheduled_maintenance_activities_model.addConstr(
            inner_start_time_var <= input_maintenance_window_end_time * inner_x_var,
            name=f'inner_maintenance_activity_start_range_con_activity_{inner_activity_id}_1'
        )
        inner_maintenance_activity_scheduling_range_cons[
            (inner_activity_id, 1, 2)] = obtain_actually_scheduled_maintenance_activities_model.addConstr(
            inner_start_time_var >= input_maintenance_window_start_time * inner_x_var,
            name=f'inner_maintenance_activity_start_range_con_activity_{inner_activity_id}_2'
        )
        inner_maintenance_activity_scheduling_range_cons[
            (inner_activity_id, 2, 1)] = obtain_actually_scheduled_maintenance_activities_model.addConstr(
            inner_end_time_var <= input_maintenance_window_end_time * inner_x_var,
            name=f'inner_maintenance_activity_end_range_con_activity_{inner_activity_id}_1'
        )
        inner_maintenance_activity_scheduling_range_cons[
            (inner_activity_id, 2, 2)] = obtain_actually_scheduled_maintenance_activities_model.addConstr(
            inner_end_time_var >= input_maintenance_window_start_time * inner_x_var,
            name=f'inner_maintenance_activity_end_range_con_activity_{inner_activity_id}_2'
        )
        inner_maintenance_activity_duration_cons[
            inner_activity_id] = obtain_actually_scheduled_maintenance_activities_model.addConstr(
            inner_end_time_var - inner_start_time_var == inner_maintenance_activity_duration * inner_x_var,  # 持续时间
            name=f'inner_maintenance_activity_duration_con_activity_{inner_activity_id}'
        )

    inner_maintenance_activities_both_scheduling_cons = {}
    for (inner_activity_id_1,
         inner_activity_id_2), inner_var in inner_maintenance_activities_both_scheduling_indicator_vars.items():
        inner_x_var_1 = inner_maintenance_activity_scheduling_vars[inner_activity_id_1]
        inner_x_var_2 = inner_maintenance_activity_scheduling_vars[inner_activity_id_2]
        inner_maintenance_activities_both_scheduling_cons[(inner_activity_id_1, inner_activity_id_2,
                                                           1)] = obtain_actually_scheduled_maintenance_activities_model.addConstr(
            inner_var <= inner_x_var_1,
            name=f'inner_maintenance_activities_both_scheduling_con_activities_{inner_activity_id_1}_{inner_activity_id_2}_1'
        )
        inner_maintenance_activities_both_scheduling_cons[(inner_activity_id_1, inner_activity_id_2,
                                                           2)] = obtain_actually_scheduled_maintenance_activities_model.addConstr(
            inner_var <= inner_x_var_2,
            name=f'inner_maintenance_activities_both_scheduling_con_activities_{inner_activity_id_1}_{inner_activity_id_2}_2'
        )
        inner_maintenance_activities_both_scheduling_cons[(inner_activity_id_1, inner_activity_id_2,
                                                           3)] = obtain_actually_scheduled_maintenance_activities_model.addConstr(
            inner_var >= inner_x_var_1 + inner_x_var_2 - 1,
            name=f'inner_maintenance_activities_both_scheduling_con_activities_{inner_activity_id_1}_{inner_activity_id_2}_3'
        )

    inner_maintenance_activities_seq_cons = {}
    for (inner_activity_id_1, inner_activity_id_2), inner_var in inner_maintenance_activities_seq_vars.items():
        inner_start_time_var_1 = inner_maintenance_activity_start_time_vars[inner_activity_id_1]
        inner_start_time_var_2 = inner_maintenance_activity_start_time_vars[inner_activity_id_2]
        inner_end_time_var_1 = inner_maintenance_activity_end_time_vars[inner_activity_id_1]
        inner_end_time_var_2 = inner_maintenance_activity_end_time_vars[inner_activity_id_2]
        inner_maintenance_activities_seq_cons[(inner_activity_id_1, inner_activity_id_2,
                                               1)] = obtain_actually_scheduled_maintenance_activities_model.addConstr(
            inner_start_time_var_2 + M * (1 - inner_var) >= inner_end_time_var_1,
            name=f'inner_maintenance_activities_seq_con_activities_{inner_activity_id_1}_{inner_activity_id_2}_1'
        )
        inner_maintenance_activities_seq_cons[(inner_activity_id_1, inner_activity_id_2,
                                               2)] = obtain_actually_scheduled_maintenance_activities_model.addConstr(
            inner_start_time_var_1 + M * inner_var >= inner_end_time_var_2,
            name=f'inner_maintenance_activities_seq_con_activities_{inner_activity_id_1}_{inner_activity_id_2}_2'
        )

    inner_major_maintenance_activity_must_be_scheduled_cons = {}
    for inner_activity_id in input_maintenance_activity_ids:
        inner_activity_type = maintenance_activities_info[inner_activity_id][0]
        if inner_activity_type == 3 or inner_activity_type == 4 or inner_activity_type == 8:
            inner_major_maintenance_activity_must_be_scheduled_cons[
                inner_activity_id] = obtain_actually_scheduled_maintenance_activities_model.addConstr(
                inner_maintenance_activity_scheduling_vars[inner_activity_id] == 1,
                name=f'inner_major_maintenance_activity_must_be_scheduled_con_activity_{inner_activity_id}'
            )

    inner_maintenance_activities_simultaneous_number_limitation_cons = {}
    for inner_activity_combination in combinations(input_maintenance_activity_ids, max_compatible_activities + 1):
        inner_activity_combination = sorted(inner_activity_combination)
        for inner_activity_id_1 in inner_activity_combination:
            for inner_activity_id_2 in inner_activity_combination:
                if inner_activity_id_2 > inner_activity_id_1:
                    if (inner_activity_id_1, inner_activity_id_2, 1) in \
                            inner_maintenance_activities_simultaneous_number_limitation_cons or (
                            inner_activity_id_1, inner_activity_id_2,
                            2) in \
                            inner_maintenance_activities_simultaneous_number_limitation_cons or (
                            inner_activity_id_1, inner_activity_id_2,
                            3) in \
                            inner_maintenance_activities_simultaneous_number_limitation_cons or (
                            inner_activity_id_1, inner_activity_id_2,
                            4) in \
                            inner_maintenance_activities_simultaneous_number_limitation_cons or (
                            inner_activity_id_1, inner_activity_id_2,
                            5) in \
                            inner_maintenance_activities_simultaneous_number_limitation_cons or (
                            inner_activity_id_1, inner_activity_id_2,
                            6) in \
                            inner_maintenance_activities_simultaneous_number_limitation_cons:
                        continue
                    if (inner_activity_id_1,
                        inner_activity_id_2) in inner_maintenance_activities_start_end_time_seq_vars and (
                            inner_activity_id_2,
                            inner_activity_id_1) in inner_maintenance_activities_start_end_time_seq_vars:
                        inner_maintenance_activities_simultaneous_number_limitation_cons[
                            (inner_activity_id_1, inner_activity_id_2,
                             1)] = obtain_actually_scheduled_maintenance_activities_model.addConstr(
                            M * (inner_maintenance_activities_start_end_time_seq_vars[
                                     (inner_activity_id_1, inner_activity_id_2)] - 1) <=
                            inner_maintenance_activity_end_time_vars[
                                inner_activity_id_2] -
                            inner_maintenance_activity_start_time_vars[
                                inner_activity_id_1],
                            name=f'inner_maintenance_activities_simultaneous_number_limitation_con_activities_{inner_activity_id_1}_{inner_activity_id_2}_1'
                        )
                        inner_maintenance_activities_simultaneous_number_limitation_cons[
                            (inner_activity_id_1, inner_activity_id_2,
                             2)] = obtain_actually_scheduled_maintenance_activities_model.addConstr(
                            inner_maintenance_activity_end_time_vars[
                                inner_activity_id_2] -
                            inner_maintenance_activity_start_time_vars[
                                inner_activity_id_1] <= M *
                            inner_maintenance_activities_start_end_time_seq_vars[
                                (inner_activity_id_1, inner_activity_id_2)],
                            name=f'inner_maintenance_activities_simultaneous_number_limitation_con_activities_{inner_activity_id_1}_{inner_activity_id_2}_2'
                        )
                        inner_maintenance_activities_simultaneous_number_limitation_cons[
                            (inner_activity_id_1, inner_activity_id_2,
                             3)] = obtain_actually_scheduled_maintenance_activities_model.addConstr(
                            M * (inner_maintenance_activities_start_end_time_seq_vars[
                                     (inner_activity_id_2, inner_activity_id_1)] - 1) <=
                            inner_maintenance_activity_end_time_vars[
                                inner_activity_id_1] -
                            inner_maintenance_activity_start_time_vars[
                                inner_activity_id_2],
                            name=f'inner_maintenance_activities_simultaneous_number_limitation_con_activities_{inner_activity_id_1}_{inner_activity_id_2}_3'
                        )
                        inner_maintenance_activities_simultaneous_number_limitation_cons[
                            (inner_activity_id_1, inner_activity_id_2,
                             4)] = obtain_actually_scheduled_maintenance_activities_model.addConstr(
                            inner_maintenance_activity_end_time_vars[
                                inner_activity_id_1] -
                            inner_maintenance_activity_start_time_vars[
                                inner_activity_id_2] <= M *
                            inner_maintenance_activities_start_end_time_seq_vars[
                                (inner_activity_id_2, inner_activity_id_1)],
                            name=f'inner_maintenance_activities_simultaneous_number_limitation_con_activities_{inner_activity_id_1}_{inner_activity_id_2}_4'
                        )
                        inner_maintenance_activities_simultaneous_number_limitation_cons[
                            (inner_activity_id_1, inner_activity_id_2,
                             5)] = obtain_actually_scheduled_maintenance_activities_model.addConstr(
                            inner_maintenance_activities_scheduling_simultaneously_indicator_vars[
                                (inner_activity_id_1, inner_activity_id_2)] ==
                            inner_maintenance_activities_start_end_time_seq_vars[
                                (inner_activity_id_1, inner_activity_id_2)] +
                            inner_maintenance_activities_start_end_time_seq_vars[
                                (inner_activity_id_2, inner_activity_id_1)] - 1,
                            name=f'inner_maintenance_activities_simultaneous_number_limitation_con_activities_{inner_activity_id_1}_{inner_activity_id_2}_5'
                        )
                        inner_maintenance_activities_simultaneous_number_limitation_cons[
                            (inner_activity_id_1, inner_activity_id_2,
                             6)] = obtain_actually_scheduled_maintenance_activities_model.addConstr(
                            inner_maintenance_activities_scheduling_simultaneously_indicator_vars[
                                (inner_activity_id_1, inner_activity_id_2)] <=
                            inner_maintenance_activities_both_scheduling_indicator_vars[
                                (inner_activity_id_1, inner_activity_id_2)],
                            name=f'inner_maintenance_activities_simultaneous_number_limitation_con_activities_{inner_activity_id_1}_{inner_activity_id_2}_6'
                        )
        if tuple(inner_activity_combination) not in \
                inner_maintenance_activities_simultaneous_number_limitation_cons:
            inner_temp_expr = gp.LinExpr(0)
            for inner_activity_id_1 in inner_activity_combination:
                for inner_activity_id_2 in inner_activity_combination:
                    if inner_activity_id_2 > inner_activity_id_1:
                        if (inner_activity_id_1,
                            inner_activity_id_2) in inner_maintenance_activities_scheduling_simultaneously_indicator_vars:
                            inner_temp_expr += \
                                inner_maintenance_activities_scheduling_simultaneously_indicator_vars[
                                    (inner_activity_id_1, inner_activity_id_2)]
            inner_maintenance_activities_simultaneous_number_limitation_cons[
                tuple(inner_activity_combination)] = obtain_actually_scheduled_maintenance_activities_model.addConstr(
                inner_temp_expr <= math.comb(max_compatible_activities + 1, 2) - 1,
                name=f'inner_maintenance_activities_simultaneous_number_limitation_con_activities_{inner_activity_combination}'
            )

    # Construct objective function
    inner_obj = gp.LinExpr(0)
    for inner_activity_id in input_maintenance_activity_ids:
        inner_importance_level = maintenance_activities_info[inner_activity_id][5]
        inner_x_var = inner_maintenance_activity_scheduling_vars[inner_activity_id]
        inner_obj += importance_penalty_costs[inner_importance_level] * inner_x_var
    obtain_actually_scheduled_maintenance_activities_model.setObjective(inner_obj, GRB.MAXIMIZE)  # The objective function is to maximize.

    obtain_actually_scheduled_maintenance_activities_model.setParam('OutputFlag', 0)
    obtain_actually_scheduled_maintenance_activities_model.setParam('TimeLimit', 30)
    obtain_actually_scheduled_maintenance_activities_model.setParam('MIPGap', 0.01)

    # Solve the model
    obtain_actually_scheduled_maintenance_activities_model.optimize()

    # Get the list of maintenance activities that are actually scheduled
    output_actually_scheduled_maintenance_activities = []
    for inner_activity_id, inner_x_var in inner_maintenance_activity_scheduling_vars.items():
        if inner_x_var.X > 0.5:  # If it is actually scheduled
            output_actually_scheduled_maintenance_activities.append(inner_activity_id)

    # Get the start and end time of actual scheduled maintenance activities
    output_actually_scheduled_maintenance_activity_start_time_dict = {
        inner_activity_id: int(inner_maintenance_activity_start_time_vars[inner_activity_id].X)
        for inner_activity_id in output_actually_scheduled_maintenance_activities
    }
    output_actually_scheduled_maintenance_activity_end_time_dict = {
        inner_activity_id: int(inner_maintenance_activity_end_time_vars[inner_activity_id].X)
        for inner_activity_id in output_actually_scheduled_maintenance_activities
    }
    return list(sorted(
        set(output_actually_scheduled_maintenance_activities))), output_actually_scheduled_maintenance_activity_start_time_dict, output_actually_scheduled_maintenance_activity_end_time_dict  # Return the deduplicated and sorted list of actually scheduled maintenance activities, as well as the dictionary of the start and end time of the actually scheduled maintenance activities

By inputting the temporary maintenance activity list after adding the unscheduled maintenance activity, obtain the shortest maintenance window duration after adding the maintenance activity

In [ ]:
def judge_whether_unscheduled_maintenance_activity_can_be_added(input_maintenance_activity_ids,
                                                                input_maintenance_window_start_time,
                                                                input_maintenance_window_duration):
    # Construct the model
    judge_whether_unscheduled_maintenance_activity_can_be_added_model = gp.Model(
        "judge_whether_unscheduled_maintenance_activity_can_be_added_model")

    # Define variables, their meanings are similar to those introduced before.
    inner_maintenance_activity_start_time_vars = {}
    inner_maintenance_activity_end_time_vars = {}
    for inner_activity_id in input_maintenance_activity_ids:
        inner_maintenance_activity_start_time_vars[
            inner_activity_id] = judge_whether_unscheduled_maintenance_activity_can_be_added_model.addVar(
            lb=input_maintenance_window_start_time,
            ub=M,
            vtype=GRB.INTEGER,
            name=f'inner_maintenance_activity_start_time_var_activity_{inner_activity_id}'
        )
        inner_maintenance_activity_end_time_vars[
            inner_activity_id] = judge_whether_unscheduled_maintenance_activity_can_be_added_model.addVar(
            lb=input_maintenance_window_start_time,
            ub=M,
            vtype=GRB.INTEGER,
            name=f'inner_maintenance_activity_end_time_var_activity_{inner_activity_id}'
        )

    inner_maintenance_activities_seq_vars = {}
    for inner_activity_id_1 in input_maintenance_activity_ids:
        inner_activity_type_1 = maintenance_activities_info[inner_activity_id_1][0]
        if inner_activity_type_1 in mutually_exclusive_activities:
            for inner_activity_id_2 in input_maintenance_activity_ids:
                if inner_activity_id_2 == inner_activity_id_1:
                    continue
                inner_activity_type_2 = maintenance_activities_info[inner_activity_id_2][0]
                if inner_activity_type_2 in mutually_exclusive_activities[inner_activity_type_1]:
                    if (inner_activity_id_1, inner_activity_id_2) in inner_maintenance_activities_seq_vars or (
                            inner_activity_id_2,
                            inner_activity_id_1) in inner_maintenance_activities_seq_vars:
                        continue
                    if inner_activity_id_1 < inner_activity_id_2:
                        inner_maintenance_activities_seq_vars[(inner_activity_id_1,
                                                               inner_activity_id_2)] = judge_whether_unscheduled_maintenance_activity_can_be_added_model.addVar(
                            vtype=GRB.BINARY,
                            name=f'inner_maintenance_activities_seq_var_activities_{inner_activity_id_1}_{inner_activity_id_2}'
                        )
                    else:
                        inner_maintenance_activities_seq_vars[(inner_activity_id_2,
                                                               inner_activity_id_1)] = judge_whether_unscheduled_maintenance_activity_can_be_added_model.addVar(
                            vtype=GRB.BINARY,
                            name=f'inner_maintenance_activities_seq_var_activities_{inner_activity_id_2}_{inner_activity_id_1}'
                        )

    inner_maintenance_activities_scheduling_simultaneously_indicator_vars = {}
    inner_maintenance_activities_start_end_time_seq_vars = {}
    for inner_activity_id_1 in input_maintenance_activity_ids:
        inner_activity_type_1 = maintenance_activities_info[inner_activity_id_1][0]
        if inner_activity_type_1 in mutually_exclusive_activities:
            for inner_activity_id_2 in input_maintenance_activity_ids:
                if inner_activity_id_2 == inner_activity_id_1:
                    continue
                inner_activity_type_2 = maintenance_activities_info[inner_activity_id_2][0]
                if inner_activity_type_2 not in mutually_exclusive_activities[inner_activity_type_1]:
                    if (inner_activity_id_1,
                        inner_activity_id_2) not in inner_maintenance_activities_scheduling_simultaneously_indicator_vars and (
                            inner_activity_id_2,
                            inner_activity_id_1) not in inner_maintenance_activities_scheduling_simultaneously_indicator_vars:
                        if inner_activity_id_1 < inner_activity_id_2:
                            inner_maintenance_activities_scheduling_simultaneously_indicator_vars[
                                (inner_activity_id_1,
                                 inner_activity_id_2)] = judge_whether_unscheduled_maintenance_activity_can_be_added_model.addVar(
                                vtype=GRB.BINARY,
                                name=f'inner_maintenance_activities_scheduling_simultaneously_indicator_var_activities_{inner_activity_id_1}_{inner_activity_id_2}'
                            )
                        else:
                            inner_maintenance_activities_scheduling_simultaneously_indicator_vars[
                                (inner_activity_id_2,
                                 inner_activity_id_1)] = judge_whether_unscheduled_maintenance_activity_can_be_added_model.addVar(
                                vtype=GRB.BINARY,
                                name=f'inner_maintenance_activities_scheduling_simultaneously_indicator_var_activities_{inner_activity_id_2}_{inner_activity_id_1}'
                            )
                    if (inner_activity_id_1,
                        inner_activity_id_2) not in inner_maintenance_activities_start_end_time_seq_vars and (
                            inner_activity_id_2,
                            inner_activity_id_1) not in inner_maintenance_activities_start_end_time_seq_vars:
                        inner_maintenance_activities_start_end_time_seq_vars[
                            (inner_activity_id_1,
                             inner_activity_id_2)] = judge_whether_unscheduled_maintenance_activity_can_be_added_model.addVar(
                            vtype=GRB.BINARY,
                            name=f'inner_maintenance_activities_start_end_time_seq_var_activities_{inner_activity_id_1}_{inner_activity_id_2}'
                        )
                        inner_maintenance_activities_start_end_time_seq_vars[
                            (inner_activity_id_2,
                             inner_activity_id_1)] = judge_whether_unscheduled_maintenance_activity_can_be_added_model.addVar(
                            vtype=GRB.BINARY,
                            name=f'inner_maintenance_activities_start_end_time_seq_var_activities_{inner_activity_id_2}_{inner_activity_id_1}'
                        )
        else:
            for inner_activity_id_2 in input_maintenance_activity_ids:
                if inner_activity_id_2 == inner_activity_id_1:
                    continue
                inner_activity_type_2 = maintenance_activities_info[inner_activity_id_2][0]
                if inner_activity_type_2 in mutually_exclusive_activities:
                    if inner_activity_type_1 in mutually_exclusive_activities[inner_activity_type_2]:
                        continue
                if (inner_activity_id_1,
                    inner_activity_id_2) not in inner_maintenance_activities_scheduling_simultaneously_indicator_vars and (
                        inner_activity_id_2,
                        inner_activity_id_1) not in inner_maintenance_activities_scheduling_simultaneously_indicator_vars:
                    if inner_activity_id_1 < inner_activity_id_2:
                        inner_maintenance_activities_scheduling_simultaneously_indicator_vars[
                            (inner_activity_id_1,
                             inner_activity_id_2)] = judge_whether_unscheduled_maintenance_activity_can_be_added_model.addVar(
                            vtype=GRB.BINARY,
                            name=f'inner_maintenance_activities_scheduling_simultaneously_indicator_var_activities_{inner_activity_id_1}_{inner_activity_id_2}'
                        )
                    else:
                        inner_maintenance_activities_scheduling_simultaneously_indicator_vars[
                            (inner_activity_id_2,
                             inner_activity_id_1)] = judge_whether_unscheduled_maintenance_activity_can_be_added_model.addVar(
                            vtype=GRB.BINARY,
                            name=f'inner_maintenance_activities_scheduling_simultaneously_indicator_var_activities_{inner_activity_id_2}_{inner_activity_id_1}'
                        )
                if (inner_activity_id_1,
                    inner_activity_id_2) not in inner_maintenance_activities_start_end_time_seq_vars and (
                        inner_activity_id_2,
                        inner_activity_id_1) not in inner_maintenance_activities_start_end_time_seq_vars:
                    inner_maintenance_activities_start_end_time_seq_vars[
                        (inner_activity_id_1,
                         inner_activity_id_2)] = judge_whether_unscheduled_maintenance_activity_can_be_added_model.addVar(
                        vtype=GRB.BINARY,
                        name=f'inner_maintenance_activities_start_end_time_seq_var_activities_{inner_activity_id_1}_{inner_activity_id_2}'
                    )
                    inner_maintenance_activities_start_end_time_seq_vars[
                        (inner_activity_id_2,
                         inner_activity_id_1)] = judge_whether_unscheduled_maintenance_activity_can_be_added_model.addVar(
                        vtype=GRB.BINARY,
                        name=f'inner_maintenance_activities_start_end_time_seq_var_activities_{inner_activity_id_2}_{inner_activity_id_1}'
                    )

    inner_maintenance_window_duration_var = judge_whether_unscheduled_maintenance_activity_can_be_added_model.addVar(
        lb=0,
        ub=M,
        vtype=GRB.CONTINUOUS,
        name='inner_maintenance_window_duration_var'
    )

    # Define constraints, their meanings are similar to those introduced before.
    inner_maintenance_activity_scheduling_range_cons = {}
    inner_maintenance_activity_duration_cons = {}

    for inner_activity_id in input_maintenance_activity_ids:
        inner_maintenance_activity_duration = maintenance_activities_info[inner_activity_id][4]
        inner_start_time_var = inner_maintenance_activity_start_time_vars[inner_activity_id]
        inner_end_time_var = inner_maintenance_activity_end_time_vars[inner_activity_id]

        inner_maintenance_activity_scheduling_range_cons[
            (inner_activity_id, 1, 1)] = judge_whether_unscheduled_maintenance_activity_can_be_added_model.addConstr(
            inner_start_time_var <= input_maintenance_window_start_time + inner_maintenance_window_duration_var,
            name=f'inner_maintenance_activity_start_range_con_activity_{inner_activity_id}_1'
        )
        inner_maintenance_activity_scheduling_range_cons[
            (inner_activity_id, 1, 2)] = judge_whether_unscheduled_maintenance_activity_can_be_added_model.addConstr(
            inner_start_time_var >= input_maintenance_window_start_time,
            name=f'inner_maintenance_activity_start_range_con_activity_{inner_activity_id}_2'
        )
        inner_maintenance_activity_scheduling_range_cons[
            (inner_activity_id, 2, 1)] = judge_whether_unscheduled_maintenance_activity_can_be_added_model.addConstr(
            inner_end_time_var <= input_maintenance_window_start_time + inner_maintenance_window_duration_var,
            name=f'inner_maintenance_activity_end_range_con_activity_{inner_activity_id}_1'
        )
        inner_maintenance_activity_scheduling_range_cons[
            (inner_activity_id, 2, 2)] = judge_whether_unscheduled_maintenance_activity_can_be_added_model.addConstr(
            inner_end_time_var >= input_maintenance_window_start_time,
            name=f'inner_maintenance_activity_end_range_con_activity_{inner_activity_id}_2'
        )
        inner_maintenance_activity_duration_cons[
            inner_activity_id] = judge_whether_unscheduled_maintenance_activity_can_be_added_model.addConstr(
            inner_end_time_var - inner_start_time_var == inner_maintenance_activity_duration,
            name=f'inner_maintenance_activity_duration_con_activity_{inner_activity_id}'
        )

    inner_maintenance_activities_seq_cons = {}
    for (inner_activity_id_1, inner_activity_id_2), inner_var in inner_maintenance_activities_seq_vars.items():
        inner_start_time_var_1 = inner_maintenance_activity_start_time_vars[inner_activity_id_1]
        inner_start_time_var_2 = inner_maintenance_activity_start_time_vars[inner_activity_id_2]
        inner_end_time_var_1 = inner_maintenance_activity_end_time_vars[inner_activity_id_1]
        inner_end_time_var_2 = inner_maintenance_activity_end_time_vars[inner_activity_id_2]
        inner_maintenance_activities_seq_cons[(inner_activity_id_1, inner_activity_id_2,
                                               1)] = judge_whether_unscheduled_maintenance_activity_can_be_added_model.addConstr(
            inner_start_time_var_2 + M * (1 - inner_var) >= inner_end_time_var_1,
            name=f'inner_maintenance_activities_seq_con_activities_{inner_activity_id_1}_{inner_activity_id_2}_1'
        )
        inner_maintenance_activities_seq_cons[(inner_activity_id_1, inner_activity_id_2,
                                               2)] = judge_whether_unscheduled_maintenance_activity_can_be_added_model.addConstr(
            inner_start_time_var_1 + M * inner_var >= inner_end_time_var_2,
            name=f'inner_maintenance_activities_seq_con_activities_{inner_activity_id_1}_{inner_activity_id_2}_2'
        )

    inner_maintenance_activities_simultaneous_number_limitation_cons = {}
    for inner_activity_combination in combinations(input_maintenance_activity_ids, max_compatible_activities + 1):
        inner_activity_combination = sorted(inner_activity_combination)
        for inner_activity_id_1 in inner_activity_combination:
            for inner_activity_id_2 in inner_activity_combination:
                if inner_activity_id_2 > inner_activity_id_1:
                    if (inner_activity_id_1, inner_activity_id_2, 1) in \
                            inner_maintenance_activities_simultaneous_number_limitation_cons or (
                            inner_activity_id_1, inner_activity_id_2,
                            2) in \
                            inner_maintenance_activities_simultaneous_number_limitation_cons or (
                            inner_activity_id_1, inner_activity_id_2,
                            3) in \
                            inner_maintenance_activities_simultaneous_number_limitation_cons or (
                            inner_activity_id_1, inner_activity_id_2,
                            4) in \
                            inner_maintenance_activities_simultaneous_number_limitation_cons or (
                            inner_activity_id_1, inner_activity_id_2,
                            5) in \
                            inner_maintenance_activities_simultaneous_number_limitation_cons or (
                            inner_activity_id_1, inner_activity_id_2,
                            6) in \
                            inner_maintenance_activities_simultaneous_number_limitation_cons:
                        continue
                    if (inner_activity_id_1,
                        inner_activity_id_2) in inner_maintenance_activities_start_end_time_seq_vars and (
                            inner_activity_id_2,
                            inner_activity_id_1) in inner_maintenance_activities_start_end_time_seq_vars:
                        inner_maintenance_activities_simultaneous_number_limitation_cons[
                            (inner_activity_id_1, inner_activity_id_2,
                             1)] = judge_whether_unscheduled_maintenance_activity_can_be_added_model.addConstr(
                            M * (inner_maintenance_activities_start_end_time_seq_vars[
                                     (inner_activity_id_1, inner_activity_id_2)] - 1) <=
                            inner_maintenance_activity_end_time_vars[
                                inner_activity_id_2] -
                            inner_maintenance_activity_start_time_vars[
                                inner_activity_id_1],
                            name=f'inner_maintenance_activities_simultaneous_number_limitation_con_activities_{inner_activity_id_1}_{inner_activity_id_2}_1'
                        )
                        inner_maintenance_activities_simultaneous_number_limitation_cons[
                            (inner_activity_id_1, inner_activity_id_2,
                             2)] = judge_whether_unscheduled_maintenance_activity_can_be_added_model.addConstr(
                            inner_maintenance_activity_end_time_vars[
                                inner_activity_id_2] -
                            inner_maintenance_activity_start_time_vars[
                                inner_activity_id_1] <= M *
                            inner_maintenance_activities_start_end_time_seq_vars[
                                (inner_activity_id_1, inner_activity_id_2)],
                            name=f'inner_maintenance_activities_simultaneous_number_limitation_con_activities_{inner_activity_id_1}_{inner_activity_id_2}_2'
                        )
                        inner_maintenance_activities_simultaneous_number_limitation_cons[
                            (inner_activity_id_1, inner_activity_id_2,
                             3)] = judge_whether_unscheduled_maintenance_activity_can_be_added_model.addConstr(
                            M * (inner_maintenance_activities_start_end_time_seq_vars[
                                     (inner_activity_id_2, inner_activity_id_1)] - 1) <=
                            inner_maintenance_activity_end_time_vars[
                                inner_activity_id_1] -
                            inner_maintenance_activity_start_time_vars[
                                inner_activity_id_2],
                            name=f'inner_maintenance_activities_simultaneous_number_limitation_con_activities_{inner_activity_id_1}_{inner_activity_id_2}_3'
                        )
                        inner_maintenance_activities_simultaneous_number_limitation_cons[
                            (inner_activity_id_1, inner_activity_id_2,
                             4)] = judge_whether_unscheduled_maintenance_activity_can_be_added_model.addConstr(
                            inner_maintenance_activity_end_time_vars[
                                inner_activity_id_1] -
                            inner_maintenance_activity_start_time_vars[
                                inner_activity_id_2] <= M *
                            inner_maintenance_activities_start_end_time_seq_vars[
                                (inner_activity_id_2, inner_activity_id_1)],
                            name=f'inner_maintenance_activities_simultaneous_number_limitation_con_activities_{inner_activity_id_1}_{inner_activity_id_2}_4'
                        )
                        inner_maintenance_activities_simultaneous_number_limitation_cons[
                            (inner_activity_id_1, inner_activity_id_2,
                             5)] = judge_whether_unscheduled_maintenance_activity_can_be_added_model.addConstr(
                            inner_maintenance_activities_scheduling_simultaneously_indicator_vars[
                                (inner_activity_id_1, inner_activity_id_2)] ==
                            inner_maintenance_activities_start_end_time_seq_vars[
                                (inner_activity_id_1, inner_activity_id_2)] +
                            inner_maintenance_activities_start_end_time_seq_vars[
                                (inner_activity_id_2, inner_activity_id_1)] - 1,
                            name=f'inner_maintenance_activities_simultaneous_number_limitation_con_activities_{inner_activity_id_1}_{inner_activity_id_2}_5'
                        )
        if tuple(inner_activity_combination) not in \
                inner_maintenance_activities_simultaneous_number_limitation_cons:
            inner_temp_expr = gp.LinExpr(0)
            for inner_activity_id_1 in inner_activity_combination:
                for inner_activity_id_2 in inner_activity_combination:
                    if inner_activity_id_2 > inner_activity_id_1:
                        if (inner_activity_id_1,
                            inner_activity_id_2) in inner_maintenance_activities_scheduling_simultaneously_indicator_vars:
                            inner_temp_expr += \
                                inner_maintenance_activities_scheduling_simultaneously_indicator_vars[
                                    (inner_activity_id_1, inner_activity_id_2)]
            inner_maintenance_activities_simultaneous_number_limitation_cons[
                tuple(
                    inner_activity_combination)] = judge_whether_unscheduled_maintenance_activity_can_be_added_model.addConstr(
                inner_temp_expr <= math.comb(max_compatible_activities + 1, 2) - 1,
                name=f'inner_maintenance_activities_simultaneous_number_limitation_con_activities_{inner_activity_combination}'
            )

    # Define the objective function
    judge_whether_unscheduled_maintenance_activity_can_be_added_model.setObjective(
        inner_maintenance_window_duration_var, GRB.MINIMIZE)  # The objective function is to minimize.

    judge_whether_unscheduled_maintenance_activity_can_be_added_model.setParam('OutputFlag', 0)
    # Set to stop when the objective function value of the feasible solution is less than or equal to input_maintenance_window_duration
    judge_whether_unscheduled_maintenance_activity_can_be_added_model.setParam('BestObjStop',
                                                                               input_maintenance_window_duration)
    judge_whether_unscheduled_maintenance_activity_can_be_added_model.setParam('TimeLimit', 30)
    judge_whether_unscheduled_maintenance_activity_can_be_added_model.setParam('MIPGap', 0.01)

    # Solve the model
    judge_whether_unscheduled_maintenance_activity_can_be_added_model.optimize()

    if int(inner_maintenance_window_duration_var.X) <= input_maintenance_window_duration:  # If the obtained maintenance window duration is less than or equal to the input maintenance window duration
        # Get the start and end time of each maintenance activity
        output_maintenance_activity_start_time_dict = {
            inner_activity_id: int(inner_maintenance_activity_start_time_vars[inner_activity_id].X)
            for inner_activity_id in input_maintenance_activity_ids
        }
        output_maintenance_activity_end_time_dict = {
            inner_activity_id: int(inner_maintenance_activity_end_time_vars[inner_activity_id].X)
            for inner_activity_id in input_maintenance_activity_ids
        }
        return True, output_maintenance_activity_start_time_dict, output_maintenance_activity_end_time_dict  # Return True, indicating that the unscheduled maintenance activity can be added, and return the start and end time of each maintenance activity.
    else:
        return False, None, None  # Returns False, indicating that the unscheduled maintenance activity cannot be added, and return an empty start time and end time dictionary

Define the function to update the Lagrangian multipliers

In [ ]:
def update_lagrangian_multipliers(input_lagrangian_multipliers,
                                  input_start_time_sol_of_maintenance_windows_for_lower_bound_solution,
                                  input_end_time_sol_of_maintenance_windows_for_lower_bound_solution,
                                  input_maintenance_activity_start_time_sol_for_lower_bound_solution,
                                  input_maintenance_activity_end_time_sol_for_lower_bound_solution,
                                  input_maintenance_activity_scheduling_day_sol_for_lower_bound_solution, input_alpha,
                                  input_global_upper_bound, input_global_lower_bound):
    # Calculate the sum of the squares of the subgradients corresponding to the lagrangian multipliers associated with constraints (21) and (22) respectively
    start_time_gradient_square_sum = 0
    end_time_gradient_square_sum = 0
    for inner_day in range(1, len(num_of_trains) + 1):
        for inner_section_id, inner_maintenance_activities in section_maintenance_activities.items():
            for inner_activity_id in inner_maintenance_activities:
                for inner_i in [1, 2]:
                    if inner_i == 1:
                        inner_maintenance_window_start_time_for_lower_bound_solution = \
                            input_start_time_sol_of_maintenance_windows_for_lower_bound_solution[inner_day][
                                inner_section_id]
                        inner_x_maintenance_activity_scheduling_day_for_lower_bound_solution = \
                            input_maintenance_activity_scheduling_day_sol_for_lower_bound_solution[inner_section_id][
                                (inner_day, inner_activity_id)]
                        inner_maintenance_activity_start_time_for_lower_bound_solution = \
                            input_maintenance_activity_start_time_sol_for_lower_bound_solution[inner_section_id][
                                (inner_day, inner_activity_id)]
                        start_time_gradient = inner_maintenance_window_start_time_for_lower_bound_solution + M * (
                                inner_x_maintenance_activity_scheduling_day_for_lower_bound_solution - 1) - inner_maintenance_activity_start_time_for_lower_bound_solution
                        start_time_gradient_square_sum += start_time_gradient ** 2
                    if inner_i == 2:
                        inner_maintenance_window_end_time_for_lower_bound_solution = \
                            input_end_time_sol_of_maintenance_windows_for_lower_bound_solution[inner_day][
                                inner_section_id]
                        inner_x_maintenance_activity_scheduling_day_for_lower_bound_solution = \
                            input_maintenance_activity_scheduling_day_sol_for_lower_bound_solution[inner_section_id][
                                (inner_day, inner_activity_id)]
                        inner_maintenance_activity_end_time_for_lower_bound_solution = \
                            input_maintenance_activity_end_time_sol_for_lower_bound_solution[inner_section_id][
                                (inner_day, inner_activity_id)]
                        # 计算次梯度
                        end_time_gradient = M * (
                                inner_x_maintenance_activity_scheduling_day_for_lower_bound_solution - 1) + inner_maintenance_activity_end_time_for_lower_bound_solution - inner_maintenance_window_end_time_for_lower_bound_solution
                        end_time_gradient_square_sum += end_time_gradient ** 2

    output_lagrangian_multipliers = copy.deepcopy(input_lagrangian_multipliers)
    for inner_day in range(1, len(num_of_trains) + 1):
        for inner_section_id, inner_maintenance_activities in section_maintenance_activities.items():
            for inner_activity_id in inner_maintenance_activities:
                for inner_i in [1, 2]:
                    if inner_i == 1:
                        inner_maintenance_window_start_time_for_lower_bound_solution = \
                            input_start_time_sol_of_maintenance_windows_for_lower_bound_solution[inner_day][
                                inner_section_id]
                        inner_x_maintenance_activity_scheduling_day_for_lower_bound_solution = \
                            input_maintenance_activity_scheduling_day_sol_for_lower_bound_solution[inner_section_id][
                                (inner_day, inner_activity_id)]
                        inner_maintenance_activity_start_time_for_lower_bound_solution = \
                            input_maintenance_activity_start_time_sol_for_lower_bound_solution[inner_section_id][
                                (inner_day, inner_activity_id)]
                        # Calculate the subgradients
                        start_time_gradient = inner_maintenance_window_start_time_for_lower_bound_solution + M * (
                                inner_x_maintenance_activity_scheduling_day_for_lower_bound_solution - 1) - inner_maintenance_activity_start_time_for_lower_bound_solution
                        new_lagrangian_multiplier = \
                            input_lagrangian_multipliers[inner_day][inner_section_id][inner_activity_id][1] + float((
                                                                                                                            input_alpha * abs(
                                                                                                                        input_global_upper_bound - input_global_lower_bound) * start_time_gradient) / start_time_gradient_square_sum)  # Update the lagrangian multipliers
                        output_lagrangian_multipliers[inner_day][inner_section_id][inner_activity_id][1] = max(
                            new_lagrangian_multiplier, 0)  # Ensure that the Lagrangian multipliers are non-negative
                    if inner_i == 2:
                        inner_maintenance_window_end_time_for_lower_bound_solution = \
                            input_end_time_sol_of_maintenance_windows_for_lower_bound_solution[inner_day][
                                inner_section_id]
                        inner_x_maintenance_activity_scheduling_day_for_lower_bound_solution = \
                            input_maintenance_activity_scheduling_day_sol_for_lower_bound_solution[inner_section_id][
                                (inner_day, inner_activity_id)]
                        inner_maintenance_activity_end_time_for_lower_bound_solution = \
                            input_maintenance_activity_end_time_sol_for_lower_bound_solution[inner_section_id][
                                (inner_day, inner_activity_id)]
                        # Calculate the subgradients
                        end_time_gradient = M * (
                                inner_x_maintenance_activity_scheduling_day_for_lower_bound_solution - 1) + inner_maintenance_activity_end_time_for_lower_bound_solution - inner_maintenance_window_end_time_for_lower_bound_solution
                        new_lagrangian_multiplier = \
                            input_lagrangian_multipliers[inner_day][inner_section_id][inner_activity_id][2] + float((
                                                                                                                            input_alpha * abs(
                                                                                                                        input_global_upper_bound - input_global_lower_bound) * end_time_gradient) / end_time_gradient_square_sum)  # Update the lagrangian multipliers
                        output_lagrangian_multipliers[inner_day][inner_section_id][inner_activity_id][2] = max(
                            new_lagrangian_multiplier, 0)  # Ensure that the Lagrangian multipliers are non-negative

    return output_lagrangian_multipliers  # Return the updated lagrangian multipliers

## The main process of the Lagrangian relaxation-based heuristic algorithm

In [ ]:
iter_num = 0  # Initialize the iteration count
max_iter_num = 50  # Set the maximum number of iterations
iter_num_gap_no_improvement = 0  # Initialize the optimal gap no-improvement iteration interval counter
best_gap = float('inf')  # Initialize the optimal gap to positive infinity
alpha = 1  # Initialize the step size coefficient
global_upper_bound = float('inf')  # Initialize the global upper bound to positive infinity
global_lower_bound = float('-inf')  # Initialize the global lower bound to negative infinity
global_upper_bound_list = []  # Initialize the global upper bound list
global_lower_bound_list = []  # Initialize the global lower bound list
current_upper_bound_list = []  # Initialize the current upper bound list
current_lower_bound_list = []  # Initialize the current lower bound list
current_gap_list = []  # Initialize the current gap list
current_best_gap_list = []  # Initialize the optimal gap list
max_time = 18000  # Set the maximum run time to 5 h
max_iter_num_gap_no_improvement = 8  # Set the maximum interval of optimal gap no-improvement iterations used to end the algorithm to 8 times
iter_termination_gap = 0.05  # Set the termination gap to 5%
iter_num_gap_no_improvement_for_updating_alpha = 3  # Set the optimal gap no-improvement iteration interval for updating the step coefficient to 3 times
best_obj1_val = 0  # Initialize the value of objective function 1 in the optimal upper bound
best_obj2_val = 0  # Initialize the value of objective function 2 in the optimal upper bound
obj1_val_list = []  # Initialize the objective function 1 value list
obj2_val_list = []  # Initialize the objective function 2 value list
iter_termination_indicator = False  # Initialize the algorithm termination indicator to False
dep_time_sol_of_actual_trains_for_lower_bound_solution = {}  # Initialize the dictionary of train departure time solutions in the lower bound solution
arr_time_sol_of_actual_trains_for_lower_bound_solution = {}  # Initialize the dictionary of train arrival time solutions in the lower bound solution
start_time_sol_of_maintenance_windows_for_lower_bound_solution = {}  # Initialize the dictionary of maintenance window start time solutions in the lower bound solution
end_time_sol_of_maintenance_windows_for_lower_bound_solution = {}  # Initialize the dictionary of maintenance window end time solutions in the lower bound solution
maintenance_activity_start_time_sol_for_lower_bound_solution = {}  # Initialize the dictionary of maintenance activity start time solutions in the lower bound solution
maintenance_activity_end_time_sol_for_lower_bound_solution = {}  # Initialize the dictionary of maintenance activity end time solutions in the lower bound solution
maintenance_activity_scheduling_day_sol_for_lower_bound_solution = {}  # Initialize the dictionary of maintenance activity scheduling days in the lower bound solution
dep_time_sol_of_actual_trains_for_upper_bound_solution = {}  # Initialize the dictionary of train departure time solutions in the upper bound solution
arr_time_sol_of_actual_trains_for_upper_bound_solution = {}  # Initialize the dictionary of train arrival time solutions in the upper bound solution
start_time_sol_of_maintenance_windows_for_upper_bound_solution = {}  # Initialize the dictionary of maintenance window start time solutions in the upper bound solution
end_time_sol_of_maintenance_windows_for_upper_bound_solution = {}  # Initialize the dictionary of maintenance window end time solutions in the upper bound solution
maintenance_activity_start_time_sol_for_upper_bound_solution = {}  # Initialize the dictionary of maintenance activity start time solutions in the upper bound solution
maintenance_activity_end_time_sol_for_upper_bound_solution = {}  # Initialize the dictionary of maintenance activity end time solutions in the upper bound solution
maintenance_activity_scheduling_day_sol_for_upper_bound_solution = {}  # Initialize the dictionary of maintenance activity scheduling days in the upper bound solution
best_dep_time_sol_of_actual_trains_for_upper_bound_solution = {}  # Initialize the dictionary of train departure time solutions in the optimal upper bound solution
best_arr_time_sol_of_actual_trains_for_upper_bound_solution = {}  # Initialize the dictionary of train arrival time solutions in the optimal upper bound solution
best_start_time_sol_of_maintenance_windows_for_upper_bound_solution = {}  # Initialize the dictionary of maintenance window start time solutions in the optimal upper bound solution
best_end_time_sol_of_maintenance_windows_for_upper_bound_solution = {}  # Initialize the dictionary of maintenance window end time solutions in the optimal upper bound solution
best_maintenance_activity_start_time_sol_for_upper_bound_solution = {}  # Initialize the dictionary of maintenance activity start time solutions in the optimal upper bound solution
best_maintenance_activity_end_time_sol_for_upper_bound_solution = {}  # Initialize the dictionary of maintenance activity end time solutions in the optimal upper bound solution
best_maintenance_activity_scheduling_day_sol_for_upper_bound_solution = {}  # Initialize the dictionary of maintenance activity scheduling days in the optimal upper bound solution
violated_station_track_capacity_con_count_reporter_for_lower_bound_solution = {}  # Initialize the dictionary of the counters for violating the station capacity-related constraint
violated_maintenance_activity_scheduling_simultaneously_con_count_reporter_for_lower_bound_solution = {}  # Initializes a dictionary of counters that violate the constraint on the number of maintenance activities to be scheduled simultaneously
violated_day_section_maintenance_activity_scheduling_simultaneously_con_count_reporter_for_upper_bound_solution = {}

In [ ]:
algorithm_start_time = time.time()  # Record the algorithm start time
while not iter_termination_indicator:
    iter_num += 1
    violated_station_track_capacity_con_count_reporter_for_lower_bound_solution[iter_num] = {}
    violated_maintenance_activity_scheduling_simultaneously_con_count_reporter_for_lower_bound_solution[
        iter_num] = {}
    violated_day_section_maintenance_activity_scheduling_simultaneously_con_count_reporter_for_upper_bound_solution[
        iter_num] = {}
    print("Iteration #", iter_num)
    if iter_num != 1:
        # If it is not the first generation, reset the model first
        for day in range(1, len(num_of_trains) + 1):
            train_timetables_with_maintenance_windows_subproblem_models[day].reset()
        for section_id in range(1, num_of_sections + 1):
            maintenance_planning_subproblem_models[section_id].reset()

        # Reset maintenance_activity_start_time_sol_for_lower_bound_solution、maintenance_activity_end_time_sol_for_lower_bound_solution、maintenance_activity_scheduling_day_sol_for_lower_bound_solution、dep_time_sol_of_actual_trains_for_upper_bound_solution、arr_time_sol_of_actual_trains_for_upper_bound_solution、start_time_sol_of_maintenance_windows_for_upper_bound_solution、end_time_sol_of_maintenance_windows_for_upper_bound_solution、maintenance_activity_start_time_sol_for_upper_bound_solution、maintenance_activity_end_time_sol_for_upper_bound_solution、maintenance_activity_scheduling_day_sol_for_upper_bound_solution
        maintenance_activity_start_time_sol_for_lower_bound_solution.clear()
        maintenance_activity_end_time_sol_for_lower_bound_solution.clear()
        maintenance_activity_scheduling_day_sol_for_lower_bound_solution.clear()
        dep_time_sol_of_actual_trains_for_upper_bound_solution.clear()
        arr_time_sol_of_actual_trains_for_upper_bound_solution.clear()
        start_time_sol_of_maintenance_windows_for_upper_bound_solution.clear()
        end_time_sol_of_maintenance_windows_for_upper_bound_solution.clear()
        maintenance_activity_start_time_sol_for_upper_bound_solution.clear()
        maintenance_activity_end_time_sol_for_upper_bound_solution.clear()
        maintenance_activity_scheduling_day_sol_for_upper_bound_solution.clear()

    # The lower bound solution
    # Define the lower bound solution
    lower_bound_total_obj_value = 0

    # The lower bound solution to subproblem related to train timetabling with maintenance window setting for each day
    # Define or update the objective function
    define_or_update_obj_of_train_timetables_with_maintenance_windows_subproblem_models()
    # For each day, call the get_lower_bound_solution_of_daily_train_timetables_with_maintenance_windows_subproblem() function in turn to solve the model of that day
    for day in range(1, len(num_of_trains) + 1):
        violated_station_track_capacity_con_count = 0
        if iter_num == 1:
            dep_time_sol_of_actual_trains_for_lower_bound_solution[day] = {}
            arr_time_sol_of_actual_trains_for_lower_bound_solution[day] = {}
            start_time_sol_of_maintenance_windows_for_lower_bound_solution[day] = {}
            end_time_sol_of_maintenance_windows_for_lower_bound_solution[day] = {}
        get_lower_bound_solution_of_daily_train_timetables_with_maintenance_windows_subproblem(day)
        daily_model = train_timetables_with_maintenance_windows_subproblem_models[day]
        if daily_model.status == GRB.OPTIMAL:
            # Get the train departure time, arrival time, maintenance window start time and end time in the lower bound solution
            for key, dep_time_var in dep_time_vars_of_actual_trains[day].items():
                dep_time_sol_of_actual_trains_for_lower_bound_solution[day][key] = int(dep_time_var.X)
            for key, arr_time_var in arr_time_vars_of_actual_trains[day].items():
                arr_time_sol_of_actual_trains_for_lower_bound_solution[day][key] = int(arr_time_var.X)
            for key, start_time_var in start_time_vars_of_maintenance_windows[day].items():
                start_time_sol_of_maintenance_windows_for_lower_bound_solution[day][key] = int(start_time_var.X)
            for key, end_time_var in end_time_vars_of_maintenance_windows[day].items():
                end_time_sol_of_maintenance_windows_for_lower_bound_solution[day][key] = int(end_time_var.X)
            lower_bound_total_obj_value += int(daily_model.ObjVal)
        else:
            for train_id, train_data in train_info[day].items():
                train_type = train_data[0]
                if train_type == 1: continue
                dep_time_sol = dep_time_sol_of_actual_trains_for_lower_bound_solution[day][(train_id, 1)]
                arr_time_sol = arr_time_sol_of_actual_trains_for_lower_bound_solution[day][(train_id, num_of_stations)]
                lower_bound_total_obj_value += w1 * (arr_time_sol - dep_time_sol)
        violated_station_track_capacity_con_count_reporter_for_lower_bound_solution[iter_num][
            day] = violated_station_track_capacity_con_count
        print(
            f'The subproblem of train timetabling with maintenance window setting for day {day} has been solved. The current violation count for the station capacity-related constraints is {violated_station_track_capacity_con_count}.')

    # The lower bound solution to subproblem related to maintenance scheduling within the planning horizon for each section
    # Define or update the objective function
    define_or_update_obj_of_maintenance_planning_subproblem_models()
    for section_id in range(1, num_of_sections + 1):
        violated_maintenance_activity_scheduling_simultaneously_con_count = 0
        maintenance_activity_start_time_sol_for_lower_bound_solution[section_id] = {}
        maintenance_activity_end_time_sol_for_lower_bound_solution[section_id] = {}
        maintenance_activity_scheduling_day_sol_for_lower_bound_solution[section_id] = {}
        # For each section, call the get_lower_bound_solution_of_maintenance_planning_subproblem() function in turn to solve the model of that section
        get_lower_bound_solution_of_maintenance_planning_subproblem(section_id)
        section_model = maintenance_planning_subproblem_models[section_id]

        for key, start_time_var in maintenance_activity_start_time_vars[section_id].items():
            maintenance_activity_start_time_sol_for_lower_bound_solution[section_id][key] = int(start_time_var.X)
        for key, end_time_var in maintenance_activity_end_time_vars[section_id].items():
            maintenance_activity_end_time_sol_for_lower_bound_solution[section_id][key] = int(end_time_var.X)
        for key, scheduling_day_var in maintenance_activity_scheduling_day_vars[section_id].items():
            maintenance_activity_scheduling_day_sol_for_lower_bound_solution[section_id][key] = int(
                scheduling_day_var.X)
        lower_bound_total_obj_value += int(section_model.ObjVal)
        violated_maintenance_activity_scheduling_simultaneously_con_count_reporter_for_lower_bound_solution[iter_num][
            section_id] = \
            violated_maintenance_activity_scheduling_simultaneously_con_count
        print(
            f'The subproblem of train timetabling with maintenance window setting for section {section_id} has been solved. The current violation count for the constraints related to the limited number of compatible maintenance activities scheduled simultaneously is {violated_maintenance_activity_scheduling_simultaneously_con_count}.')

    # Add a constant term
    for day in range(1, len(num_of_trains) + 1):
        for section_id, maintenance_activities in section_maintenance_activities.items():
            for activity_id in maintenance_activities:
                lagrangian_multiplier_of_start = lagrangian_multipliers[day][section_id][activity_id][1]
                lagrangian_multiplier_of_end = lagrangian_multipliers[day][section_id][activity_id][2]
                lower_bound_total_obj_value -= M * (lagrangian_multiplier_of_start + lagrangian_multiplier_of_end)

    # Update the optimal lower bound value
    if lower_bound_total_obj_value > global_lower_bound:
        global_lower_bound = lower_bound_total_obj_value

    # Update the global lower bound list
    global_lower_bound_list.append(global_lower_bound)
    current_lower_bound_list.append(lower_bound_total_obj_value)  # 更新当前下界列表
    print(f'The lower bound solution is generated. The lower bound value of the current iteration is {lower_bound_total_obj_value}, and the global lower bound is {global_lower_bound}.')
    print('===========================================================================')

    # The upper bound solution
    upper_bound_total_obj_value = 0
    obj1_val = 0

    dep_time_sol_of_actual_trains_for_upper_bound_solution = copy.deepcopy(
        dep_time_sol_of_actual_trains_for_lower_bound_solution)  # Directly use the departure times of the lower bound solution as the departure times of the upper bound solution
    arr_time_sol_of_actual_trains_for_upper_bound_solution = copy.deepcopy(
        arr_time_sol_of_actual_trains_for_lower_bound_solution)  # Directly use the arrival times of the lower bound solution as the arrival times of the upper bound solution
    for day, day_train_info in train_info.items():
        for train_id, train_data in day_train_info.items():
            train_type = train_data[0]
            if train_type == 1: continue
            dep_time_sol = dep_time_sol_of_actual_trains_for_upper_bound_solution[day][(train_id, 1)]
            arr_time_sol = arr_time_sol_of_actual_trains_for_upper_bound_solution[day][(train_id, num_of_stations)]
            upper_bound_total_obj_value += w1 * (arr_time_sol - dep_time_sol)
            obj1_val += (arr_time_sol - dep_time_sol)
    print(f'The value of objective function 1 in the upper bound solution is {int(obj1_val)}.')

    # Then repair the maintenance scheduling plan
    # Step 1: Extend the maintenance window for each section of the day
    for day, day_train_info in train_info.items():
        start_time_sol_of_maintenance_windows_for_upper_bound_solution[day] = {}
        end_time_sol_of_maintenance_windows_for_upper_bound_solution[day] = {}
        for i in range(1, num_of_sections + 1):
            maintenance_window_start_time = start_time_sol_of_maintenance_windows_for_lower_bound_solution[day][i]
            maintenance_window_end_time = end_time_sol_of_maintenance_windows_for_lower_bound_solution[day][i]
            # Initialize the ID of the immediately preceding and following trains of the section maintenance window
            adjacent_before_train_id = None
            adjacent_after_train_id = None
            # Initialize the arrival time of the preceding train at the end station of the section to 0, and initialize the departure time of the following train at the start station of the section to M
            adjacent_before_train_arr_time = 0
            adjacent_after_train_dep_time = M

            for train_id, train_data in day_train_info.items():
                train_arr_time = arr_time_sol_of_actual_trains_for_upper_bound_solution[day][(train_id, i + 1)]
                train_dep_time = dep_time_sol_of_actual_trains_for_upper_bound_solution[day][(train_id, i)]

                if train_arr_time <= maintenance_window_start_time:
                    if train_arr_time >= adjacent_before_train_arr_time:
                        adjacent_before_train_id = train_id
                        adjacent_before_train_arr_time = train_arr_time

                if train_dep_time >= maintenance_window_end_time:
                    if train_dep_time <= adjacent_after_train_dep_time:
                        adjacent_after_train_id = train_id
                        adjacent_after_train_dep_time = train_dep_time

            # After finding and updating the IDs and times of the immediately preceding and following trains, extend the maintenance window as much as possible
            # Extend forward to the rear of the preceding train and meet the minimum arrival-departure headway requirements between the train and the maintenance window
            if adjacent_before_train_id is not None:
                adjacent_before_train_type = day_train_info[adjacent_before_train_id][0]
                if adjacent_before_train_type == 1:
                    new_maintenance_window_start_time = adjacent_before_train_arr_time + had_p
                    start_time_sol_of_maintenance_windows_for_upper_bound_solution[day][
                        i] = new_maintenance_window_start_time
                else:
                    new_maintenance_window_start_time = adjacent_before_train_arr_time + had_f
                    start_time_sol_of_maintenance_windows_for_upper_bound_solution[day][
                        i] = new_maintenance_window_start_time
            # Extended backward to the front of the immediately following train and meet the minimum arrival-departure headway requirement between the train and the maintenance window
            if adjacent_after_train_id is not None:
                adjacent_after_train_type = day_train_info[adjacent_after_train_id][0]
                if adjacent_after_train_type == 1:
                    new_maintenance_window_end_time = adjacent_after_train_dep_time - had_p
                    end_time_sol_of_maintenance_windows_for_upper_bound_solution[day][
                        i] = new_maintenance_window_end_time
                else:
                    new_maintenance_window_end_time = adjacent_after_train_dep_time - had_f
                    end_time_sol_of_maintenance_windows_for_upper_bound_solution[day][
                        i] = new_maintenance_window_end_time
    print('Step 1 of the upper bound solution generation is completed, and the maintenance window for each section of each day has been extended.')

    # Step 2：
    # 1. Traverse the maintenance activities in each section, construct the set of scheduled maintenance activities in each section every day in the lower bound solution and fill it into 'scheduled_maintenance_activities_each_day_each_section_in_lower_bound_solution', and construct the set of unscheduled maintenance activities in each section in the lower bound solution and fill it into 'unscheduled_maintenance_activities_each_section_in_lower_bound_solution'
    scheduled_maintenance_activities_each_day_each_section_in_lower_bound_solution = {}
    unscheduled_maintenance_activities_each_section_in_lower_bound_solution = {}
    for section_id in range(1, num_of_sections + 1):
        unscheduled_maintenance_activities_each_section_in_lower_bound_solution[section_id] = []  # 初始化每个区间未调度的维修活动列表
        for day in range(1, len(num_of_trains) + 1):
            scheduled_maintenance_activities_each_day_each_section_in_lower_bound_solution[
                (day, section_id)] = []

    for section_id, maintenance_activities in section_maintenance_activities.items():
        for activity_id in maintenance_activities:
            whether_scheduled = False
            for day in range(1, len(num_of_trains) + 1):
                x_sol = maintenance_activity_scheduling_day_sol_for_lower_bound_solution[section_id][(day, activity_id)]
                if x_sol > 0.5:
                    scheduled_maintenance_activities_each_day_each_section_in_lower_bound_solution[
                        (day, section_id)].append(
                        activity_id)
                    whether_scheduled = True
            if not whether_scheduled:
                unscheduled_maintenance_activities_each_section_in_lower_bound_solution[section_id].append(activity_id)
                print(
                    f'In the lower bound solution, the maintenance activity {activity_id} is not scheduled on any day and is added to the list of unscheduled maintenance activities for the section {section_id}.')

    # 2. Traverse the maintenance activities of each section of each day from 'scheduled_maintenance_activities_each_day_each_section_in_lower_bound_solution', and according to the start and end time (duration) of the maintenance window of the section on that day, obtain the maintenance activities actually scheduled for each section of each day and fill in 'actually_scheduled_maintenance_activities_each_day_each_section_in_upper_bound_solution' and the unscheduled maintenance activities and fill in 'unscheduled_maintenance_activities_each_day_each_section_in_upper_bound_solution'
    actually_scheduled_maintenance_activities_each_day_each_section_in_upper_bound_solution = {}
    unscheduled_maintenance_activities_each_day_each_section_in_upper_bound_solution = {}
    for (day,
         section_id), maintenance_activity_ids in scheduled_maintenance_activities_each_day_each_section_in_lower_bound_solution.items():
        day_section_violated_maintenance_activity_scheduling_simultaneously_con_count = 0
        maintenance_window_start_time = start_time_sol_of_maintenance_windows_for_upper_bound_solution[day][section_id]
        maintenance_window_end_time = end_time_sol_of_maintenance_windows_for_upper_bound_solution[day][section_id]
        # Input 'maintenance_activity_ids', 'maintenance_window_start_time' and 'maintenance_window_end_time' into the function obtain_actually_scheduled_maintenance_activities() to output the list of maintenance activities that can actually be scheduled in this section on that day
        actually_scheduled_maintenance_activities, actually_scheduled_maintenance_activity_start_time_dict, actually_scheduled_maintenance_activity_end_time_dict = obtain_actually_scheduled_maintenance_activities(
            sorted(maintenance_activity_ids), maintenance_window_start_time, maintenance_window_end_time)
        # Add the list of maintenance activities actually scheduled to the set of maintenance activities actually scheduled for each section of each day in the upper bound solution
        actually_scheduled_maintenance_activities_each_day_each_section_in_upper_bound_solution[
            (day, section_id)] = actually_scheduled_maintenance_activities
        # Add the start and end times of the maintenance activities actually scheduled to the dictionary of maintenance activity start and end times in the upper bound solution
        for activity_id, start_time in actually_scheduled_maintenance_activity_start_time_dict.items():
            maintenance_activity_start_time_sol_for_upper_bound_solution[(day, activity_id)] = start_time
        for activity_id, end_time in actually_scheduled_maintenance_activity_end_time_dict.items():
            maintenance_activity_end_time_sol_for_upper_bound_solution[(day, activity_id)] = end_time
        for activity_id in actually_scheduled_maintenance_activities:
            maintenance_activity_scheduling_day_sol_for_upper_bound_solution[(day, activity_id)] = 1
        # The difference between 'maintenance_activity_ids' and 'actually_scheduled_maintenance_activities' is 'unscheduled_maintenance_activities'
        unscheduled_maintenance_activities = list(
            set(maintenance_activity_ids) - set(actually_scheduled_maintenance_activities))
        unscheduled_maintenance_activities_each_day_each_section_in_upper_bound_solution[
            (day, section_id)] = unscheduled_maintenance_activities
        violated_day_section_maintenance_activity_scheduling_simultaneously_con_count_reporter_for_upper_bound_solution[
            iter_num][(day, section_id)] = day_section_violated_maintenance_activity_scheduling_simultaneously_con_count
        print(f'Obtain the actually scheduled maintenance activities for section {section_id} on day {day}')
    print('Step 2 of the upper bound solution generation is completed, and the actually scheduled maintenance activities and unscheduled maintenance activities have been generated for each day and each section.')

    # Step 3：
    # 1.First, process 'unscheduled_maintenance_activities_each_section_in_lower_bound_solution' and 'unscheduled_maintenance_activities_each_day_each_section_in_upper_bound_solution', and construct a new dictionary 'unscheduled_maintenance_activities_each_section' to store the unscheduled maintenance activities and their information for each section. Its outer key is the section ID, the inner key is the maintenance activity ID, and the inner value is the maintenance activity information.
    unscheduled_maintenance_activities_each_section = {}
    for section_id in range(1, num_of_sections + 1):
        unscheduled_maintenance_activities_each_section[section_id] = {}
        # Process the unscheduled maintenance activities for each day in the current section in 'unscheduled_maintenance_activities_each_day_each_section_in_upper_bound_solution'
        for day in range(1, len(num_of_trains) + 1):
            unscheduled_maintenance_activities = \
                unscheduled_maintenance_activities_each_day_each_section_in_upper_bound_solution[(day, section_id)]
            if len(unscheduled_maintenance_activities) == 0:
                continue
            for unscheduled_activity_id in unscheduled_maintenance_activities:
                earliest_scheduling_day = maintenance_activities_info[unscheduled_activity_id][1]
                latest_scheduling_day = maintenance_activities_info[unscheduled_activity_id][2]
                importance_level = maintenance_activities_info[unscheduled_activity_id][5]
                unscheduled_maintenance_activities_each_section[section_id][unscheduled_activity_id] = [
                    day, earliest_scheduling_day, latest_scheduling_day, importance_level]
        # Process the unscheduled maintenance activities for each day in the current section in 'unscheduled_maintenance_activities_each_day_each_section_in_lower_bound_solution'
        if len(unscheduled_maintenance_activities_each_section_in_lower_bound_solution[section_id]) != 0:
            for unscheduled_activity_id in unscheduled_maintenance_activities_each_section_in_lower_bound_solution[
                section_id]:
                earliest_scheduling_day = maintenance_activities_info[unscheduled_activity_id][1]
                latest_scheduling_day = maintenance_activities_info[unscheduled_activity_id][2]
                importance_level = maintenance_activities_info[unscheduled_activity_id][5]
                unscheduled_maintenance_activities_each_section[section_id][unscheduled_activity_id] = [
                    0, earliest_scheduling_day, latest_scheduling_day, importance_level]

    # 2.Sort the unscheduled maintenance activities in each section in 'unscheduled_maintenance_activities_each_section' by importance level from large to small, and obtain a list of maintenance activities in each section by importance level.
    unscheduled_maintenance_activities_importance_sorted_each_section = {}
    for section_id, unscheduled_maintenance_activities in unscheduled_maintenance_activities_each_section.items():
        if len(unscheduled_maintenance_activities) == 0:
            continue
        importance_sorted_list = sorted(unscheduled_maintenance_activities.keys(),
                                        key=lambda x: unscheduled_maintenance_activities[x][3],  # 通过activity_id查重要性
                                        reverse=True)
        unscheduled_maintenance_activities_importance_sorted_each_section[section_id] = importance_sorted_list

    # Process the unscheduled maintenance activities for each section in 'unscheduled_maintenance_activities_importance_sorted_each_section'
    for section_id, sorted_unscheduled_maintenance_activities in unscheduled_maintenance_activities_importance_sorted_each_section.items():
        if len(sorted_unscheduled_maintenance_activities) == 0:
            continue
        for unscheduled_activity_id in sorted_unscheduled_maintenance_activities:
            scheduling_day_in_lower_bound_solution = \
                unscheduled_maintenance_activities_each_section[section_id][unscheduled_activity_id][0]
            earliest_scheduling_day = \
                unscheduled_maintenance_activities_each_section[section_id][unscheduled_activity_id][1]
            latest_scheduling_day = \
                unscheduled_maintenance_activities_each_section[section_id][unscheduled_activity_id][2]
            for other_day in range(1, len(num_of_trains) + 1):
                if other_day == scheduling_day_in_lower_bound_solution or not (
                        earliest_scheduling_day <= other_day <= latest_scheduling_day):
                    continue
                other_day_maintenance_window_start_time = \
                    start_time_sol_of_maintenance_windows_for_upper_bound_solution[other_day][section_id]
                other_day_maintenance_window_end_time = \
                    end_time_sol_of_maintenance_windows_for_upper_bound_solution[other_day][section_id]
                other_day_maintenance_window_duration = other_day_maintenance_window_end_time - other_day_maintenance_window_start_time
                other_day_actually_scheduled_maintenance_activities = \
                    actually_scheduled_maintenance_activities_each_day_each_section_in_upper_bound_solution[
                        (other_day, section_id)]
                # Set up a temporary maintenance activity list, add 'unscheduled_activity_id' and 'other_day_scheduled_maintenance_activities' to the list, and operations on the list will not affect other_day_scheduled_maintenance_activities
                temp_scheduled_maintenance_activities = copy.deepcopy(
                    other_day_actually_scheduled_maintenance_activities)
                temp_scheduled_maintenance_activities.append(unscheduled_activity_id)
                indicator_whether_unscheduled_maintenance_activity_can_be_added, maintenance_activity_start_time_dict, maintenance_activity_end_time_dict = judge_whether_unscheduled_maintenance_activity_can_be_added(
                    sorted(temp_scheduled_maintenance_activities), other_day_maintenance_window_start_time,
                    other_day_maintenance_window_duration)
                print(
                    f'Judge whether the unscheduled maintenance activity {unscheduled_activity_id} can be added in the section {section_id} of day {other_day}. The result is {indicator_whether_unscheduled_maintenance_activity_can_be_added}.')
                # If 'indicator_whether_unscheduled_maintenance_activity_can_be_added' is True, it means that the unscheduled maintenance activity can be added.
                if indicator_whether_unscheduled_maintenance_activity_can_be_added:
                    actually_scheduled_maintenance_activities_each_day_each_section_in_upper_bound_solution[
                        (other_day, section_id)].append(unscheduled_activity_id)
                    unscheduled_maintenance_activities_each_section[section_id].pop(unscheduled_activity_id, None)
                    for activity_id, start_time in maintenance_activity_start_time_dict.items():
                        maintenance_activity_start_time_sol_for_upper_bound_solution[
                            (other_day, activity_id)] = start_time
                    for activity_id, end_time in maintenance_activity_end_time_dict.items():
                        maintenance_activity_end_time_sol_for_upper_bound_solution[(other_day, activity_id)] = end_time
                    for activity_id in temp_scheduled_maintenance_activities:
                        maintenance_activity_scheduling_day_sol_for_upper_bound_solution[(other_day, activity_id)] = 1
                    break
    print('Step 3 of the upper bound solution generation is completed, and the unscheduled maintenance activities has been adjusted.')

    obj2_val = 0
    # Finally, check the remaining maintenance activity ID key-value pairs in each section in 'unscheduled_maintenance_activities_each_section'. These maintenance activities are the unscheduled maintenance activities, and calculate the total upper bound objective value based on their importance level
    for section_id, unscheduled_maintenance_activities in unscheduled_maintenance_activities_each_section.items():
        if len(unscheduled_maintenance_activities) == 0:
            continue
        for unscheduled_activity_id, activity_info in unscheduled_maintenance_activities.items():
            importance_level = activity_info[3]
            upper_bound_total_obj_value += w2 * importance_penalty_costs[importance_level]
            obj2_val += importance_penalty_costs[importance_level]
    print(f'The value of objective function 2 in the upper bound solution is {int(obj2_val)}.')

    # Update the optimal upper bound target value and the optimal upper bound solution
    if upper_bound_total_obj_value < global_upper_bound:
        global_upper_bound = upper_bound_total_obj_value
        best_arr_time_sol_of_actual_trains_for_upper_bound_solution = copy.deepcopy(
            arr_time_sol_of_actual_trains_for_upper_bound_solution)
        best_dep_time_sol_of_actual_trains_for_upper_bound_solution = copy.deepcopy(
            dep_time_sol_of_actual_trains_for_upper_bound_solution)
        best_start_time_sol_of_maintenance_windows_for_upper_bound_solution = copy.deepcopy(
            start_time_sol_of_maintenance_windows_for_upper_bound_solution)
        best_end_time_sol_of_maintenance_windows_for_upper_bound_solution = copy.deepcopy(
            end_time_sol_of_maintenance_windows_for_upper_bound_solution)
        best_maintenance_activity_start_time_sol_for_upper_bound_solution = copy.deepcopy(
            maintenance_activity_start_time_sol_for_upper_bound_solution)
        best_maintenance_activity_end_time_sol_for_upper_bound_solution = copy.deepcopy(
            maintenance_activity_end_time_sol_for_upper_bound_solution)
        best_maintenance_activity_scheduling_day_sol_for_upper_bound_solution = copy.deepcopy(
            maintenance_activity_scheduling_day_sol_for_upper_bound_solution)
        best_obj1_val = obj1_val
        best_obj2_val = obj2_val

    # Update the global upper bound list
    global_upper_bound_list.append(global_upper_bound)
    current_upper_bound_list.append(upper_bound_total_obj_value)  # 更新当前上界列表
    obj1_val_list.append(obj1_val)  # 更新当前目标函数1值列表
    obj2_val_list.append(obj2_val)  # 更新当前目标函数2值列表
    print(f'The upper bound solution is generated. The upper bound value of the current iteration is {upper_bound_total_obj_value}, and the global upper bound is {global_upper_bound}.')
    print('===========================================================================')

    current_best_gap = abs(global_upper_bound - global_lower_bound) / global_upper_bound
    current_best_gap_list.append(current_best_gap)
    current_gap = abs(upper_bound_total_obj_value - lower_bound_total_obj_value) / upper_bound_total_obj_value
    current_gap_list.append(current_gap)
    # Update optimal gap
    if current_best_gap < best_gap:
        best_gap = current_best_gap
        iter_num_gap_no_improvement = 0
    else:
        iter_num_gap_no_improvement += 1
    print(
        f'The gap of the current iteration is {current_gap}, the best gap is {current_best_gap}, the best gap of all iterations is {best_gap}, and the counter of the number of iterations without improvement of the best gap is {iter_num_gap_no_improvement}.')

    algorithm_current_time = time.time()
    algorithm_elapsed_time = algorithm_current_time - algorithm_start_time
    print(f'The current running time is {algorithm_elapsed_time:.2f} s.')

    # Judge whether the algorithm termination condition is reached
    if iter_num >= max_iter_num or iter_num_gap_no_improvement >= max_iter_num_gap_no_improvement or algorithm_elapsed_time >= max_time:
        iter_termination_indicator = True
    else:
        # Update Lagrangian multipliers
        # Judge whether the step coefficient needs to be updated
        if iter_num_gap_no_improvement >= iter_num_gap_no_improvement_for_updating_alpha:
            alpha = float(alpha / 2)
        lagrangian_multipliers = update_lagrangian_multipliers(lagrangian_multipliers,
                                                               start_time_sol_of_maintenance_windows_for_lower_bound_solution,
                                                               end_time_sol_of_maintenance_windows_for_lower_bound_solution,
                                                               maintenance_activity_start_time_sol_for_lower_bound_solution,
                                                               maintenance_activity_end_time_sol_for_lower_bound_solution,
                                                               maintenance_activity_scheduling_day_sol_for_lower_bound_solution,
                                                               alpha, global_upper_bound, global_lower_bound)
        print(f'The Lagrangian multipliers are updated, and the current step coefficient is {alpha}.')
        print('===========================================================================')
        print()
        print()

## Draw up daily train timetables, maintenance windows and maintenance plans

In [ ]:
print(f'The optimal value of objective function 1 is {best_obj1_val}.')
print(f'The optimal value of objective function 2 is {best_obj2_val}.')

In [ ]:
importance_unscheduled_activities_num = {importance_level: 0 for importance_level in importance_penalty_costs.keys()}

for section_id, maintenance_activities in section_maintenance_activities.items():
    for activity_id in maintenance_activities:
        is_scheduled_indicator = False
        for day in range(1, len(num_of_trains) + 1):
            if (day, activity_id) in best_maintenance_activity_start_time_sol_for_upper_bound_solution and \
                    (day, activity_id) in best_maintenance_activity_end_time_sol_for_upper_bound_solution:
                is_scheduled_indicator = True
                break
        if not is_scheduled_indicator:
            activity_importance_level = maintenance_activities_info[activity_id][5]
            importance_unscheduled_activities_num[activity_importance_level] += 1

print(f'The number of unscheduled maintenance activities at each importance level is {importance_unscheduled_activities_num}.')

In [ ]:
# Obtain the final daily train timetables
finally_obtained_daily_train_timetables = {}
for day, daily_train_info in train_info.items():
    finally_obtained_daily_train_timetables[day] = {}
    for train_id, train_data in daily_train_info.items():
        finally_obtained_daily_train_timetables[day][train_id] = []
        for station_id in range(1, num_of_stations + 1):
            if station_id == 1:
                dep_time = best_dep_time_sol_of_actual_trains_for_upper_bound_solution[day][(train_id, station_id)]
                finally_obtained_daily_train_timetables[day][train_id].append((dep_time, station_id))
            elif station_id == num_of_stations:
                arr_time = best_arr_time_sol_of_actual_trains_for_upper_bound_solution[day][(train_id, station_id)]
                finally_obtained_daily_train_timetables[day][train_id].append((arr_time, station_id))
            else:
                arr_time = best_arr_time_sol_of_actual_trains_for_upper_bound_solution[day][(train_id, station_id)]
                dep_time = best_dep_time_sol_of_actual_trains_for_upper_bound_solution[day][(train_id, station_id)]
                finally_obtained_daily_train_timetables[day][train_id].append((arr_time, station_id))
                finally_obtained_daily_train_timetables[day][train_id].append((dep_time, station_id))

In [ ]:
# Obtain the final daily maintenance windows
finally_obtained_daily_maintenance_windows = {}
for day in train_info:
    finally_obtained_daily_maintenance_windows[day] = {}
    d_start_time_of_maintenance_windows = best_start_time_sol_of_maintenance_windows_for_upper_bound_solution[day]
    d_end_time_of_maintenance_windows = best_end_time_sol_of_maintenance_windows_for_upper_bound_solution[day]
    for section_id in d_start_time_of_maintenance_windows:
        finally_obtained_daily_maintenance_windows[day][section_id] = [[d_start_time_of_maintenance_windows[section_id],
                                                                        d_end_time_of_maintenance_windows[section_id]],
                                                                       [section_id, section_id + 1]]

In [ ]:
# Obtain the scheduling time of maintenance activities
finally_obtained_daily_maintenance_activity_slots = {}
for day in train_info:
    finally_obtained_daily_maintenance_activity_slots[day] = {}
    for section_id, maintenance_activities in section_maintenance_activities.items():
        for activity_id in maintenance_activities:
            if (day, activity_id) in best_maintenance_activity_start_time_sol_for_upper_bound_solution and \
                    (day, activity_id) in best_maintenance_activity_end_time_sol_for_upper_bound_solution:
                start_time = best_maintenance_activity_start_time_sol_for_upper_bound_solution[(day, activity_id)]
                end_time = best_maintenance_activity_end_time_sol_for_upper_bound_solution[(day, activity_id)]
                finally_obtained_daily_maintenance_activity_slots[day][activity_id] = [[start_time, end_time],
                                                                                       [section_id, section_id + 1]]

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(15, 15), sharex=False, sharey=True)

# The distances between adjacent stations
station_distances = [78, 67.9, 67.9, 67.5, 75.6, 50.8]  # Unit: km

station_names = {1: 'WC', 2: 'XN', 3: 'ZLQ', 4: 'YY', 5: 'ML', 6: 'CS', 7: 'ZZ'}

cumulative_distances = [0]
for dist in station_distances:
    cumulative_distances.append(cumulative_distances[-1] + dist)

min_dist = min(cumulative_distances)
max_dist = max(cumulative_distances)
normalized_distances = [1 + 6 * (d - min_dist) / (max_dist - min_dist) for d in cumulative_distances]

for day, ax in zip([1, 2, 3], axes):
    ax.set_yticks(normalized_distances)
    ax.set_yticklabels([station_names[i] for i in range(1, len(station_names) + 1)])
    ax.set_ylim(normalized_distances[0] - 0.5, normalized_distances[-1] + 0.5)

    ax.set_xlim(0, 16 * 60)

    ax.xaxis.set_major_locator(plt.MultipleLocator(60))
    ax.xaxis.set_minor_locator(plt.MultipleLocator(30))


    def time_formatter(x, pos=None):
        total_minutes = int(x)
        if x < 360:
            hours = 18 + total_minutes // 60
        else:
            hours = total_minutes // 60 - 6
        return f"{hours}:00"


    ax.xaxis.set_major_formatter(plt.FuncFormatter(time_formatter))

    ax.set_xticks(range(0, 16 * 60 + 1, 60))
    plt.setp(ax.get_xticklabels(), visible=True)

    plt.setp(ax.get_xticklabels(), rotation=45, ha='right')

    ax.grid(True, which='major', linestyle='-', alpha=0.7)
    ax.grid(True, which='minor', linestyle=':', alpha=0.5)

    ax.set_ylabel("Station")
    ax.set_xlabel("Time")

    ax.text(0.01, 0.95, f"Day {day}", transform=ax.transAxes,
            fontsize=12, bbox=dict(facecolor='white', alpha=0.8))

    daily_train_timetables = finally_obtained_daily_train_timetables[day]
    for train_id, timetable in daily_train_timetables.items():
        train_type = train_info[day][train_id][0]
        color = 'red' if train_type == 1 else 'blue'

        for i in range(len(timetable) - 1):
            start_time, start_station = timetable[i]
            end_time, end_station = timetable[i + 1]

            ax.plot([start_time, end_time],
                    [normalized_distances[start_station - 1], normalized_distances[end_station - 1]],
                    color=color, linewidth=1.5, alpha=0.8)

    daily_maintenance_windows = finally_obtained_daily_maintenance_windows[day]
    for section_id in daily_maintenance_windows:
        vertices = [
            [daily_maintenance_windows[section_id][0][0],
             normalized_distances[daily_maintenance_windows[section_id][1][0] - 1]],
            [daily_maintenance_windows[section_id][0][1],
             normalized_distances[daily_maintenance_windows[section_id][1][0] - 1]],
            [daily_maintenance_windows[section_id][0][1],
             normalized_distances[daily_maintenance_windows[section_id][1][1] - 1]],
            [daily_maintenance_windows[section_id][0][0],
             normalized_distances[daily_maintenance_windows[section_id][1][1] - 1]]
        ]

        rect = patches.Polygon(
            vertices,
            closed=True,
            fill=True,
            facecolor="lightgray",
            edgecolor="darkgray",
            linewidth=2,
            alpha=0.8
        )

        ax.add_patch(rect)

    daily_maintenance_activity_slots = finally_obtained_daily_maintenance_activity_slots[day]
    for activity_id in daily_maintenance_activity_slots:
        maintenance_info = maintenance_activities_info[activity_id]
        activity_type = maintenance_info[0]
        if activity_type == 3 or activity_type == 4 or activity_type == 8:
            color = 'yellow'
        else:
            color = 'saddlebrown'
        ax.plot(daily_maintenance_activity_slots[activity_id][0],
                [normalized_distances[daily_maintenance_activity_slots[activity_id][1][0] - 1],
                 normalized_distances[daily_maintenance_activity_slots[activity_id][1][1] - 1]],
                color=color, linewidth=1.5, alpha=0.8)

plt.tight_layout()

if balanced_maintenance_indicator == 1:
    file_name_for_whether_balanced = "balanced"
else:
    file_name_for_whether_balanced = "unbalanced"

if maintenance_volume_indicator == 1:
    file_name_for_maintenance_volume = '180_maintenance_volume'
else:
    file_name_for_maintenance_volume = '240_maintenance_volume'

if maintenance_mutually_exclusive_indicator == 1:
    file_name_for_mutually_exclusion = '29_mutually_exclusion'
else:
    file_name_for_mutually_exclusion = '39_mutually_exclusion'

svg_name = f'{w1}_{w2}_train_timetables_with_trains_{num_of_trains[0]}_{num_of_trains[1]}_{num_of_trains[2]}_{file_name_for_whether_balanced}_{file_name_for_maintenance_volume}_{file_name_for_mutually_exclusion}_max_compatible_num_{max_compatible_activities}.svg'

plt.savefig(svg_name, format='svg', dpi=1200, bbox_inches='tight')

plt.show()

## Other results analysis

Output the optimal upper bound and optimal lower bound as the number of iterations changes

In [ ]:
iterations_0 = list(range(1, len(global_lower_bound_list) + 1))

plt.figure(figsize=(10, 6))

plt.plot(iterations_0, global_lower_bound_list, label='Global Lower Bound', color='#6baed6', marker='o')
plt.plot(iterations_0, global_upper_bound_list, label='Global Upper Bound', color='#fb6a4a', marker='s')

plt.xlabel('Iteration Number', fontsize=12)
plt.ylabel('Bound Value', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)

plt.xlim(min(iterations_0) - 0.5, max(iterations_0) + 2)

plt.xticks(range(int(min(iterations_0)), int(max(iterations_0)) + 1))

plt.legend(fontsize=12)

plt.tight_layout()

plt.savefig(
    f'{w1}_{w2}_gap_curve_with_trains_{num_of_trains[0]}_{num_of_trains[1]}_{num_of_trains[2]}_{file_name_for_whether_balanced}_{file_name_for_maintenance_volume}_{file_name_for_mutually_exclusion}_max_compatible_num_{max_compatible_activities}.svg',
    format='svg', dpi=1200, bbox_inches='tight')

plt.show()

Constraint violations in the lower bound solution

In [ ]:
iterations_1 = sorted(violated_station_track_capacity_con_count_reporter_for_lower_bound_solution.keys())
keys_1 = list(range(1, len(num_of_trains) + 1))

series_1 = {}
for k_1 in keys_1:
    series_1[k_1] = [violated_station_track_capacity_con_count_reporter_for_lower_bound_solution[it_1][k_1] for it_1
                     in
                     iterations_1]

averages_1 = {k_1: round(np.mean(v_1)) for k_1, v_1 in series_1.items()}

plt.figure(figsize=(20, 6))

colors_1 = ['#8c96c6', '#8856a7', '#810f7c']
for k_1 in keys_1:
    plt.plot(iterations_1, series_1[k_1], label=f'Day {k_1}: {averages_1[k_1]} (avg)',
             color=colors_1[k_1 - 1], marker='o', linestyle='-')

for k_1 in keys_1:
    plt.axhline(y=averages_1[k_1], color=colors_1[k_1 - 1], linestyle='--', alpha=0.5)
    if k_1 == 1:
        plt.text(iterations_1[-1] + 0.5, averages_1[k_1], averages_1[k_1],
                 color=colors_1[k_1 - 1], va='top', ha='left')
    else:
        plt.text(iterations_1[-1] + 0.5, averages_1[k_1], averages_1[k_1],
                 color=colors_1[k_1 - 1], va='bottom', ha='left')

plt.xlabel('Iteration Number', fontsize=12)
plt.ylabel('The number of violated constraints', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.5)

plt.xlim(min(iterations_1) - 0.5, max(iterations_1) + 2)

plt.xticks(range(int(min(iterations_1)), int(max(iterations_1)) + 1))

plt.legend(fontsize=12, loc='upper right')

plt.tight_layout()

plt.savefig(
    f'{w1}_{w2}_violated_cons_number_of_train_timetabling_subproblems_with_trains_{num_of_trains[0]}_{num_of_trains[1]}_{num_of_trains[2]}_{file_name_for_whether_balanced}_{file_name_for_maintenance_volume}_{file_name_for_mutually_exclusion}_max_compatible_num_{max_compatible_activities}.svg',
    format='svg', dpi=1200, bbox_inches='tight')

plt.show()

In [ ]:
iterations_2 = sorted(
    violated_maintenance_activity_scheduling_simultaneously_con_count_reporter_for_lower_bound_solution.keys())
keys_2 = list(range(1, num_of_sections + 1))

series_2 = {}
for k_2 in keys_2:
    series_2[k_2] = [
        violated_maintenance_activity_scheduling_simultaneously_con_count_reporter_for_lower_bound_solution[it_2][k_2]
        for it_2 in iterations_2]

averages_2 = {k_2: round(np.mean(v_2)) for k_2, v_2 in series_2.items()}

plt.figure(figsize=(20, 8))

colors_2 = ['#c7e9b4', '#7fcdbb', '#41b6c4', '#1d91c0', '#225ea8', '#0c2c84']
for k_2 in keys_2:
    plt.plot(iterations_2, series_2[k_2], label=f'Section {k_2}: {averages_2[k_2]} (avg)',
             color=colors_2[k_2 - 1], marker='o', linestyle='-')

for k_2 in keys_2:
    plt.axhline(y=averages_2[k_2], color=colors_2[k_2 - 1], linestyle='--', alpha=0.5)
    if k_2 in [3, 4, 5]:
        plt.text(iterations_2[-1] + 0.5, averages_2[k_2], averages_2[k_2],
                 color=colors_2[k_2 - 1], va='top', ha='left')
    else:
        plt.text(iterations_2[-1] + 0.5, averages_2[k_2], averages_2[k_2],
                 color=colors_2[k_2 - 1], va='bottom', ha='left')

plt.xlabel('Iteration Number', fontsize=12)
plt.ylabel('The number of violated constraints', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.5)
plt.xlim(min(iterations_2) - 0.5, max(iterations_2) + 2)

plt.xticks(range(int(min(iterations_2)), int(max(iterations_2)) + 1))

plt.legend(fontsize=12, loc='upper right')

plt.tight_layout()

plt.savefig(
    f'{w1}_{w2}_violated_cons_number_of_maintenance_planning_subproblems_with_trains_{num_of_trains[0]}_{num_of_trains[1]}_{num_of_trains[2]}_{file_name_for_whether_balanced}_{file_name_for_maintenance_volume}_{file_name_for_mutually_exclusion}_max_compatible_num_{max_compatible_activities}.svg',
    format='svg', dpi=1200, bbox_inches='tight')

plt.show()

In [ ]:
day_section_maintenance_window_duration_and_maintenance_volume = {}

for day in range(1, len(num_of_trains) + 1):
    day_section_maintenance_window_duration_and_maintenance_volume[day] = {}
    for section_id, maintenance_activities in section_maintenance_activities.items():
        num_of_day_section_maintenance_activities = 0
        for activity_id in maintenance_activities:
            if (day, activity_id) in best_maintenance_activity_scheduling_day_sol_for_upper_bound_solution:
                num_of_day_section_maintenance_activities += 1
        day_section_maintenance_window_duration_and_maintenance_volume[day][section_id] = [
            best_end_time_sol_of_maintenance_windows_for_upper_bound_solution[day][section_id] -
            best_start_time_sol_of_maintenance_windows_for_upper_bound_solution[day][section_id],
            num_of_day_section_maintenance_activities]

In [ ]:
days = list(range(1, len(num_of_trains) + 1))
sections = list(range(1, num_of_sections + 1))

daily_avg = {
    day: {
        'duration': round(
            np.mean([day_section_maintenance_window_duration_and_maintenance_volume[day][sec][0] for sec in sections])),
        'volume': round(
            np.mean([day_section_maintenance_window_duration_and_maintenance_volume[day][sec][1] for sec in sections]))
    }
    for day in days
}

day_colors = ['#8c96c6', '#8856a7', '#810f7c']
section_colors = ['#c7e9b4', '#7fcdbb', '#41b6c4', '#1d91c0', '#225ea8', '#0c2c84']

fig, ax = plt.subplots(figsize=(16, 8))

ax.axhline(y=0, color='darkgrey', linewidth=3)
ax.set_xlim(-1, len(days) * (num_of_sections + 2) - 1)

max_duration = max(
    max(day_section_maintenance_window_duration_and_maintenance_volume[day][sec][0] for sec in sections) for day in
    days)
max_volume = max(
    max(day_section_maintenance_window_duration_and_maintenance_volume[day][sec][1] for sec in sections) for day in
    days)
scale_factor = max_duration / max_volume
scaled_max_volume = round(max_volume * scale_factor)

ax.set_ylim(-scaled_max_volume * 1.1, max_duration * 1.1)

bar_width = 0.6
for day_idx, day in enumerate(days):
    day_x_start = day_idx * (num_of_sections + 2)

    for sec_idx, sec in enumerate(sections):
        x_pos = day_x_start + sec_idx
        duration = day_section_maintenance_window_duration_and_maintenance_volume[day][sec][0]
        volume = round(
            day_section_maintenance_window_duration_and_maintenance_volume[day][sec][1] * scale_factor)

        ax.bar(x_pos, duration, width=bar_width,
               color=section_colors[sec_idx], alpha=0.8,
               edgecolor='white', linewidth=0.7)

        ax.bar(x_pos, -volume, width=bar_width,
               color=section_colors[sec_idx], alpha=0.8,
               edgecolor='white', linewidth=0.7)

        ax.text(x_pos, duration + max_duration * 0.02, duration,
                ha='center', va='bottom', fontsize=9)
        ax.text(x_pos, -volume - scaled_max_volume * 0.02,
                day_section_maintenance_window_duration_and_maintenance_volume[day][sec][1],
                ha='center', va='top', fontsize=9)

    avg_x = day_x_start + num_of_sections
    avg_duration = daily_avg[day]['duration']
    avg_volume = round(daily_avg[day]['volume'] * scale_factor)

    ax.bar(avg_x, avg_duration, width=bar_width,
           color=day_colors[day_idx], alpha=0.8,
           edgecolor='white', linewidth=0.7)
    ax.bar(avg_x, -avg_volume, width=bar_width,
           color=day_colors[day_idx], alpha=0.8,
           edgecolor='white', linewidth=0.7)

    ax.text(avg_x, avg_duration + max_duration * 0.02, avg_duration,
            ha='center', va='bottom', fontsize=9, fontweight='bold')
    ax.text(avg_x, -avg_volume - scaled_max_volume * 0.02, daily_avg[day]['volume'],
            ha='center', va='top', fontsize=9, fontweight='bold')

    ax.text(day_x_start + (num_of_sections - 1 + 2) / 2 - 0.5, -scaled_max_volume * 1.05, f'Day {day}',
            ha='center', va='top', fontsize=12, fontweight='bold',
            bbox=dict(facecolor='white', alpha=0.8, edgecolor='none'))

for day_idx in range(1, len(days)):
    x_pos = day_idx * (num_of_sections + 2) - 1
    ax.axvline(x=x_pos, color='gray', linestyle=':', alpha=0.5)

ax.set_ylabel('Maintenance window duration', fontsize=12)

duration_ticks = np.linspace(0, 200, 6)
volume_ticks = np.linspace(0, 20, 6)

ax.yaxis.set_ticks(duration_ticks)
ax.yaxis.set_ticklabels([f'{int(y)}' for y in duration_ticks])

ax2 = ax.twinx()
ax2.set_ylim(-scaled_max_volume * 1.1, max_duration * 1.1)
ax2.yaxis.set_ticks(- np.round(volume_ticks * scale_factor))
ax2.yaxis.set_ticklabels([f'{int(y)}' for y in volume_ticks])
ax2.set_ylabel('Maintenance volume', fontsize=12)

ax.set_xticks([])

section_legends = [
    plt.Rectangle((0, 0), 1, 1, fc=section_colors[i], alpha=0.8,
                  edgecolor='white', linewidth=0.7)
    for i in range(num_of_sections)
]
section_labels = [f'Section {i + 1}' for i in range(num_of_sections)]

avg_legends = [
    plt.Rectangle((0, 0), 1, 1, fc=day_colors[i], alpha=0.8,
                  edgecolor='white', linewidth=0.7)
    for i in range(len(days))
]
avg_labels = [f'Avg value of day {i + 1}' for i in range(len(days))]

legends = section_legends + avg_legends
labels = section_labels + avg_labels

ax.legend(legends, labels, loc='upper center',
          bbox_to_anchor=(0.5, 1.05), ncol=9, fontsize=10)

plt.tight_layout()

plt.savefig(
    f'{w1}_{w2}_maintenance_window_duration_vs_maintenance_volume_with_trains_{num_of_trains[0]}_{num_of_trains[1]}_{num_of_trains[2]}_{file_name_for_whether_balanced}_{file_name_for_maintenance_volume}_{file_name_for_mutually_exclusion}_max_compatible_num_{max_compatible_activities}.svg',
    format='svg', dpi=1200, bbox_inches='tight')

plt.show()

In [ ]:
section_heights = [78, 67.9, 67.9, 67.5, 75.6, 50.8]

fig, axs = plt.subplots(6, 3, figsize=(18, 12),
                        gridspec_kw={
                            'height_ratios': section_heights[::-1],
                            'hspace': 0.3,
                            'wspace': 0.1
                        })

for section in range(1, num_of_sections + 1):
    for day in range(1, len(num_of_trains) + 1):
        ax = axs[6 - section, day - 1]

        for spine in ax.spines.values():
            spine.set_linewidth(3)
            spine.set_color('darkgray')

        maintenance_window_duration = finally_obtained_daily_maintenance_windows[day][section][0][1] - \
                                      finally_obtained_daily_maintenance_windows[day][section][0][0]
        ax.set_xlim(0, maintenance_window_duration)
        ax.set_ylim(0, section_heights[section - 1])

        ax.set_xticks([0, maintenance_window_duration])
        ax.set_xticklabels(['0', f'{maintenance_window_duration}'], fontsize=9)

        ax.set_yticks([])

        ax.set_title(f'Section {section} - Day {day}', fontsize=12, pad=10)

        for activity_id in section_maintenance_activities[section]:
            if activity_id in finally_obtained_daily_maintenance_activity_slots[day]:
                start_time = finally_obtained_daily_maintenance_activity_slots[day][activity_id][0][0] - \
                             finally_obtained_daily_maintenance_windows[day][section][0][0]
                end_time = finally_obtained_daily_maintenance_activity_slots[day][activity_id][0][1] - \
                           finally_obtained_daily_maintenance_windows[day][section][0][0]
                maintenance_info = maintenance_activities_info[activity_id]
                activity_type = maintenance_info[0]
                if activity_type == 3 or activity_type == 4 or activity_type == 8:
                    color = 'yellow'
                else:
                    color = 'saddlebrown'
                ax.plot([start_time, end_time],
                        [0, section_heights[section - 1]], color=color, linewidth=2, alpha=0.8)

plt.subplots_adjust(hspace=0.4, wspace=0.2)

plt.savefig(
    f'section_day_maintenance_plan_with_trains_{num_of_trains[0]}_{num_of_trains[1]}_{num_of_trains[2]}_{file_name_for_whether_balanced}_{file_name_for_maintenance_volume}_{file_name_for_mutually_exclusion}_max_compatible_num_{max_compatible_activities}.svg',
    format='svg', dpi=1200, bbox_inches='tight')

plt.show()

In [ ]:
# Calculate the utilization rate of maintenance window in each section and each day
section_day_maintenance_window_utilization_rate = {}
for section in range(1, num_of_sections + 1):
    for day in range(1, len(num_of_trains) + 1):
        maintenance_window_duration = finally_obtained_daily_maintenance_windows[day][section][0][1] - \
                                      finally_obtained_daily_maintenance_windows[day][section][0][0]
        latest_end_time = 0
        for activity_id in section_maintenance_activities[section]:
            if activity_id in finally_obtained_daily_maintenance_activity_slots[day]:
                end_time = finally_obtained_daily_maintenance_activity_slots[day][activity_id][0][1] - \
                           finally_obtained_daily_maintenance_windows[day][section][0][0]
                if end_time > latest_end_time:
                    latest_end_time = end_time
        section_day_maintenance_window_utilization_rate[(day, section)] = round(latest_end_time / maintenance_window_duration, 4)

In [ ]:
average_utilization_rate = sum(section_day_maintenance_window_utilization_rate.values()) / len(
    section_day_maintenance_window_utilization_rate)
print(f'Average utilization rate: {average_utilization_rate:.4f}.')